In [1]:
# 17_search.ipynb

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/app')

In [2]:
from hybrid_search import hybrid_search

In [3]:
from qdrant_search import QdrantSearchEngine
from bm25_search import ElasticsearchBM25
voyage_apikey = 'pa-g4x1yHL4Hv6qBjXcrT2Nhm5gG0srlDhVBp6Op7_IJBQ'


qdrant_engine = QdrantSearchEngine(voyage_apikey)
es_engine = ElasticsearchBM25()

Connecting to Elasticsearch at elasticsearch:9200


In [57]:
q = 'collect tasks and action points that i promised to do and other peoplele promeiced to me so far'

In [58]:
r = await hybrid_search(q, qdrant_engine, es_engine)

In [59]:
import pandas as pd
df = pd.DataFrame(r['results']).sort_values('meeting_id', ascending=False)

In [60]:
def format_meeting_context(df):
    # Create meeting_id to integer mapping
    unique_meetings = df['meeting_id'].unique()
    meeting_map = {mid: idx+1 for idx, mid in enumerate(unique_meetings)}
    meeting_map_reverse = {v:k for k,v in meeting_map.items()}
    
    # Sort by timestamp and meeting_id
    df = df.sort_values(['timestamp', 'meeting_id'])
    
    meetings = []
    for meeting_id, group in df.groupby('meeting_id'):
        # Get integer meeting id
        int_meeting_id = meeting_map[meeting_id]
        
        # Format timestamp
        timestamp = pd.to_datetime(group['timestamp'].iloc[0])
        date_header = f"## Meeting {int_meeting_id} - {timestamp.strftime('%B %d, %Y %H:%M')}"
        
        # Format topic if available
        topic = group['topic'].iloc[0]
        if topic:
            date_header += f"\nTopic: {topic}"
            
        # Collect content with formatted time
        content_items = []
        for _, row in group.iterrows():
            time_prefix = f"[{row['formatted_time']}]" if row['formatted_time'] else ''
            content_line = f"- {time_prefix} {row['content']}"
            content_items.append(content_line)
        
        content = "\n".join(content_items)
        meetings.append(f"{date_header}\n\n{content}\n")
    
    return "\n".join(meetings), meeting_map_reverse

# Create formatted context and store mapping
input_context, meeting_id_map = format_meeting_context(df)

In [61]:
from core import generic_call,system_msg,user_msg,count_tokens

In [62]:
input_context

"## Meeting 28 - November 25, 2024 13:11\nTopic: Introduction and Overview\n\n- [00:00] Dmitriy Grankin:  Запускаемся и смотрим, что там такое происходит.\n- [00:39] Dmitriy Grankin:  Пускай мы все и сможем.\n\n## Meeting 27 - December 02, 2024 12:10\nTopic: Обсуждение услуг и комьюнити\n\n- [30:12] Dmitriy Grankin:  Там эти регаты, тусовки, там по 80 евро собрали людей во время санкций.  там чем-то немножко покормили кальян и поставили окей я вообще не против все нормально глубины нет в силу того что Она там не предполагается. Я когда Antler проходил, я тоже очень удивился, что я как технический фаундер, как бы мои квалификации, вообще что я думаю.  по поводу того как реализовывать ту или иную фичу вообще проект никого не  интересует абсолютно на этом этапе на этапе присед это типа вообще никого не интересует  странная мысль но тем не менее так это работает вот короче не знаю пока что  делать но мне вроде хочется тоже чуть-чуть зацепить эту ситуацию но  торчать 50 баксов вообще этим д

In [63]:
from IPython.display import display, Markdown
display(Markdown(input_context))


## Meeting 28 - November 25, 2024 13:11
Topic: Introduction and Overview

- [00:00] Dmitriy Grankin:  Запускаемся и смотрим, что там такое происходит.
- [00:39] Dmitriy Grankin:  Пускай мы все и сможем.

## Meeting 27 - December 02, 2024 12:10
Topic: Обсуждение услуг и комьюнити

- [30:12] Dmitriy Grankin:  Там эти регаты, тусовки, там по 80 евро собрали людей во время санкций.  там чем-то немножко покормили кальян и поставили окей я вообще не против все нормально глубины нет в силу того что Она там не предполагается. Я когда Antler проходил, я тоже очень удивился, что я как технический фаундер, как бы мои квалификации, вообще что я думаю.  по поводу того как реализовывать ту или иную фичу вообще проект никого не  интересует абсолютно на этом этапе на этапе присед это типа вообще никого не интересует  странная мысль но тем не менее так это работает вот короче не знаю пока что  делать но мне вроде хочется тоже чуть-чуть зацепить эту ситуацию но  торчать 50 баксов вообще этим детским садом  заниматься, мне не хочется. Мне не хочется  понимать вообще, на что я подписываюсь. Вообще  не знаю. Посмотрим. Вот.  Все, рассказала Кубин. Давай,  погнали, поработаем. Давай, давай.  Спасибо за время.
- [02:15] Tatiana Sukhova:  Так, в каком порядке хочешь? Смотри, у меня есть один быстренький UX improvement, потом у меня есть идея и у меня есть, ну, вот то, что спросил.  Ну, давай.  В таком порядке и будет.  Окей, ладно.  Я сегодня начинаю новую работу фулл-тайм, поэтому мне надо себя, понимаешь, показывать изо всех сил.  Что за работа?  Дизайн-лид в стартапе американском Wine Distribution Industry.  Я просто была вынуждена схватить самую первую попавшуюся.  потому что я не была готова сейчас искать долго новую, но ничего, есть у меня там прямо классно отработанный план, который я хочу отработать, которые можно потом применять вообще на любые стартапы.  Вот. Ладно. Значит, UX.  Значит, помимо того, что мы с тобой уже обсудили, я хочу...  предложить что-то поделать с вот этой табличкой, потому что она съедает вертикальное пространство, и вообще весь фокус должен быть вот на этом поле.  Это поле, оно должно быть самым значимым элементом вообще в позиции всей.  Его надо явно выделить ярче, вот как сейчас, видишь, обводкой даже он светится, он как бы привлекает все меня.  Поднять чуть-чуть повыше, чтобы его не пересекали никакие другие элементы всплывающие.  Вот сейчас у меня, например, я не знаю, ты видишь, это у меня есть гугл-таблицка, а надо на нее нажать.  И вертикальный вот этот space самого разговора с чатом, он самый ценный.  То есть мы его должны максимально освободить.  чего-либо но это хочу там ты когда будешь чатам пользоваться у тебя будут эти чаты появляться Я думаю, что надо эту историю там показать, чтобы можно было использовать повторно чаты.  Вот, и да, ну и всякие пресеты.  Пресеты, то есть те, которые ты будешь использовать, добавлять.  А история же может в вертикале храниться.  Я, например, когда пользуюсь чатом GPS.  Вот смотрите, сейчас прямо покажу, как я им пользуюсь.  Вот, допустим, я в чат GPT.  У меня здесь несколько кастомных GPT на тренингах, с которыми я взаимодействую.  Вот один, например, RAR Direction.  RAR Direction я использую...  Для того, чтобы туда запускаю скриншот стиля, и он мне пишет, короче говоря, по определенной схеме все данные, которые мне надо.  Никогда нету возможности вернуться.  Я вот, знаешь, грубо говоря, переключилась на другой чат, потому что мне надо было срочно промп, или потому что мне надо было срочно какой-то текст для UI вот в этих проектах.  И...  А оттуда и с того я еще не все взяла, что мне надо было.  Это жутко неудобно.  То есть вот чтобы здесь вот оно просто оставалось всегда.  И, ну, как, знаешь, как в любом мессенджере.  И когда мне просто надо вспомнить про разговор Ну, какой-то диалог с этим чатом трехдневной дальности или недельной дальности.  Их можно как-то коллапс делать, то есть группировать для раскрытия.  Но я могу просто тупо...  скроллить обратно вверх и найти все, что мне нужно.  Ну, они вообще у тебя тут есть слева.  Видишь, сегодня Free Tokens for Recraft.  Вот.  Вот.  Но это уже даже если...  Если ты не знала об этом, это больше их проблема, что ты об этом не знала, чем твоя.  А остальные? Вот, допустим, вчера?  И остальные тоже.  Слушай, действительно есть, прикинь, я не знала об этом.  Это просто раньше гораздо появилось, чем эти кастомные чаты, поэтому...  Офигеть.  Слушай, нет, ну это разные чаты, да?  А я говорю про то, что...  Видишь, этот у меня другой, как бы per chat.  То есть вот представь, что я как будто с разными...  Внутри проекта, да, который находится?  Да, внутри проекта.  Кастом GPT, который внутри кастом GPT.  Да, да.  Вот мне было бы удобно так, потому что я их использую.  А ты сейчас Вексу запустила, а что она тебе новую встречу не купит?  Потому что это у меня в рабочем профайле вот эта Векса.  А, это другой профайл.  Да, то есть я здесь с другими людьми встречаюсь, и я, блин, в пятницу не записала такой важный разговор, я была уверена, что записывала.  Серьезно?  И сегодня я должна была к нему вернуться, потому что он там быстро много говорит на английском, я просто не успеваю.  Все, это граф. Я сегодня должна была вернуться, у меня первый пункт, блин, врубиться в его вообще концепцию.  Захожу, нету. Я такая, блядь. Ну, может, я не нажала. Знаешь, тоже не исключено.  Угу, вот дела. Вот это вот, да. То есть, с одной стороны, видишь, людям нравится то, что он не заходит как отдельный бот. С другой стороны, обычно...  Аккуратная сторона этого бонуса это то, что ты должен нажать кнопочку, если не нажмешь, то не запишется.  Ну да, и Data Privacy меня очень беспокоит, потому что я вообще вот не имею права это все записывать.  Ну у тебя же там, он же отправляет сообщения.  Сейчас отправляет.  но у меня контракт подписан, что я, в принципе, вообще не должна этого делать.  По-хорошему, я должна прекратить это делать и вообще не делать.  Но поскольку мне это очень удобно, я пока просто не запоминаю, что мне говорят, Мне приходится сейчас это делать, потому что мне это реально помогает.  Ну, надо посмотреть, что ты можешь, что ты не можешь.  Вопрос записывать – это одна история.  ты же не записываешь ничего у тебя нет записи не на где-то же это хранится  транскрипт вот этих разговоров да а завод а запись не вот это же где-то хранится все в текстовом виде и это не у них хранится и не у меня хранится это еще где-то  хранится это проблема ну вот я эту проблему вижу вижу его еще и как вам Возможность в том смысле, что, в принципе, вот этот весь бэкэнд, где хранятся данные, обрабатываются данные, это, в принципе, может быть как контейнер, Который можно разместить в контуре компании, и я не буду иметь доступа к данным вообще.  И реально огромное количество...  Число компаний, которые хотели бы, но не могут использовать такие инструменты или, может быть, используют, но беспокоятся по поводу данных.  Они, может быть, доверяют OpenAI, много кому доверяют, но, например, стартапу не доверяют.  Да-да-да, конечно, однозначно.  В общем, смотри, потом про вот эту я говорила, я имею в виду на клике.  показывать расширять еще вот этот вот фрейм вот этот борда вниз прямо под пилами показывать вот этот текст еще было бы хорошо провести connection между вот этим текстом и вот этим То есть, допустим, я на него кликнула, он стал черным, контрастным, прям reverse, inverse.  Вот этот текст под вот этими всеми пилами появился карточкой, черная тоже с этим.  текстом если я кликнула вот этот пил например первый он стал черным и вот эта карточка черным то есть чтобы они сконнектились эти две штуки а вот так когда я открываю она мне перекрывает  Я не могу на них кликать.  Ты хочешь, чтобы были...  Смысл в том, чтобы были видны остальные, да?  Да, чтобы были видны остальные, но тоже было...  И это тоже видно.  То есть я клацаю по пилам, у меня меняется текст...  внизу. Классу второй раз, unselect, убирается карточка с текстом внизу тоже. Понимаешь, да?  Проделилась, убралась,  но также по такому принципу. Но чатность  Смотрю, ты ни разу не использовала вообще ни в каком виде.  В рабочем нет еще.  А, в рабочем нет.  Вот.  Еще нет.  Я только заступаю, во-первых.  Вот здесь...  использовала здесь использовала много здесь вот я вышли наш у меня мешает вот эта фигня потому что мало остается для текста вот проблема в том что очень много привирает.  То есть вот эта история, конечно, она пишет от себя, Дина, очень много. Это  прямо плохо.  Вот, то есть я вот, смотри, я тебе хотела как раз показать, вот так Выглядит таблица CastDev, которую мне выдали в этом акселераторе.  Вот вопросы.  Вот они.  И вот мне тут надо заполнять было...
- [11:49] Tatiana Sukhova:  записывать структурные ответы, структурные, видимо, по номерам, и flow мысленный, то есть прямо параграфами эту тему. Представляешь, какая это Адская работа. Это просто идиотизм. Я уже вторую неделю не могу это заполнить, потому что у меня тупо нет времени, если это автоматизировать. И вот тут мне пришла идея в голову.  ну да то есть это все надо ковать пока горячо вот ты такой поговорил с человеком у тебя там все сохранилось причем ты должен доверять потому что ну ты пока помнишь что я вижу что тут как бы пока копирую хожу вексу вопросы вот эти вот и задаю я вижу что он приплетает просто вообще процентов 80 понимаешь вот у него структура выдать 5 пунктов вот он тебе выдаст 5 пунктов понимаешь было там вообще не было какие-то вопросы я пропускаю потому что я чувствую что они очень релевантно вообще с этим конкретно  человеком вот посмотрим кое-что какой ты режим используешь насколько ты понимаешь как чат работает А что тут разные режимы, где тут режимы?  Ну вот этот вот, вот эта плашка, да, которая сингл митинг, она понятна, что она хочет сказать или не понятна?  Нет, я вообще говорю, вот зачем эта плашка?  Она тут только место занимает.  Я уже ее тыкаю вообще во все стороны.  Не понимаю, зачем она тут.  Непонятно, да, что она делает.  Ничего не кликается.  Да, здесь просто...  У чата есть как минимум два режима. Один, который работает по выборке, по отфильтрованным встречам.  всем сразу вообще по всем своим встречам что спрашивает а есть сингл митинг сингл митинг он на вход получает транскрипт а А Multi, он получает на вход другие структуры, и там сложная система подачи.  ...контекста. Вот, соответственно, непонятно, да, то есть просто какая-то плашка, которая мешает, не более того.  Вижу, да, вижу.  И в этом смысле...  Абсолютно непонятно.  Абсолютно непонятно.  Ну, слушай, это все можно порешать, это все как бы UI, UX детали.  Пока она съедает вертикально.  И то, что переключать между этими режимами, ну как бы это все надо делать где-то гораздо более компактно, в горизонтали, нежели вертикально съедать.  Вот.  Значит, все, я тебе рассказала, табличку показала.  Я просто, мне так обидно, потому что я время потратила.  Я не могу никак на следующий этап перейти в этом, блин, акселераторе.  мне надо сдать это задание я не знаю когда я найду еще время и я уже забываю вот эти вообще первые все интервью еще немножко помню про третье короче самое пока полезное для У меня было это перечитать транскрипт самой, чтобы вспомнить прямо, как разговор у нас шел, и тут копировать отсюда, так сказать, уже действительно моменты, которые я как человек, Вижу, отвечают моим целям построения нового продукта.  Вопросы для вас.  А когда обидно тебе, а что тебе именно обидно?  Обидно, что я потратила время на эти интервью, ты же людей нашел, ты с ними списался, потом они забукали, не забукали, наконец забукали.  потом ты это время выделил, когда ты full-time занят, и когда у тебя вообще куча каких-то проектов несется параллельно, это прямо инвестиция хорошая. Потом ты с ними поговорил, энергию свою потратил, потом ты это все начинаешь решать.  перерабатывать вручную я выходные сидела понимаешь я уже не в том возрасте чтобы выходные прям сидеть мне надо отдыхать выходные вот и вот меня становится обидно что я несмотря на все эти потраченные усилия я так и не довела до конца цель не достигла а давай  попробуем взять какой-нибудь вопрос какую-нибудь встречу и  Ну вот все, что ты здесь видишь заполнено, это все так и было сделано.  Вот последний третий уже я лучше всего.  То есть я уже тут какие-то мысли попробовала покопировать из транскрипта и структурировано вот вопрос копировала и копировала ответы.  как у тебя флоу выглядел, если ты с чатом пытаешься что-то сделать? – Ну вот, Берона привезли вопрос. Сначала я все это с русского  на английский перевела на русском русском русские ребята запилили этот акселератор но я на русском вообще не воспринимали я не могу вообще на русском все не работает мозг вот здесь я отправляю у меня тут вот эта фигня какая-то блокирует меня вообще ну вот вот так вот я смотрела тут всю воду  все что как-то релевантно я вот так вот копировала и вставляла смотри видишь у тебя вот этот режим он здесь вообще совсем не к месту потому что одна встреча выдачи но он все равно мульти ну как бы ну а почему ну ну просто потому что это дополнительная разработка и все просто все упирается в ресурсы вот попробую вы вы открыть его этот чат Не чат, а встречу. Видишь, сменилась тема. И, например, лучше открой другую, новую, new conversation.  как попробуй сюда дропнуть лучше будет или нет свой запас это как бы но уже за Уже поняв, проблемы все ясны, мы их вычленили, теперь уже могу тебе подсказать.  Ну, это все равно гораздо больше, чем то, что она мне сообщила, я тебе так скажу.  То есть вообще, я заметила, недостаточно сложно вытягивать информацию, то есть она мне, допустим, сказала от силы 2,5 вообще каких-то frustrations или что-то там, challenges, вот, а он тут, конечно, поднапридумывал вообще согласно темы. Согласно тем, что он сам  об этом думает.  А ты можешь попробовать его допромкнуть так, чтобы оно сработало?  Это, ну, для чего? Для того, чтобы, например, я мог потом... У тебя, например, получится сейчас докрутить, а я потом возьму твой промт и...  И его в системный превращу, и мы попробуем...
- [00:00] :  Привет, Таня.  Привет.  Как настроение?  Привет. Как настроение? Огонь, настроение огонь. Класс. Вчера что-то было не очень, я поняла, что я об людей подразрядилась за выходные. Много встретила новых людей, которых не знала до этого, и меня это немножко подразрядило.

## Meeting 26 - December 06, 2024 12:05
Topic: User customization and preferences

- [05:14] Dmitriy Grankin:  Yeah, same thing after and this so now can you set your own preferences as a user right I'm not a technical user I'm a project manager working in a company and I want to set my preferences It's like, okay, I want to get these summaries, I want to get a list of actions.  Yes.  And then I want to get them all in Portuguese or whatever.  and um suggest me some answers or something but all all the time those are the preferences that i want to get and get them in my email yes it's just in the pipeline Next thing, because it's like all the users say the same thing consistently.  So it will be the next thing.  I will play with it.  Go ahead.
- [06:54] Dmitriy Grankin:  have you have you used teams teams before and for not yeah uh microsoft teams yeah not yet unfortunately it has a feature like this right so it does exactly same thing however it doesn't do the suggested actions and the it doesn't do a couple of things but what will stop them from doing this in uh in teams and in slack i think about nothing would stop us it's like when people ask about competition i'd say these situations like imagine a bus like later in the ...evening and you have like a capacity of 80 people in the bus and you have two people in the bus and it stops and another two people come. That's the daily situation on the It's very sparse. There is like a lot of interest and a lot of curiosity, some competitors, but it's pretty much like...  we don't even know you know which is 80% let's say and then the other the other 20 percent 50 percent of them they use slack and they both have the same features slack has it's working on this and they're they're gonna do it even better better whatever every time features and all that and it's not only you but they have 300 engineers doing the working or not uh and then uh teams has not only their own copilot issues but they the whole OpenAI engineering team, and they get all the features first.  They get 5.0, 6.0, 7.0 GPT access two months in advance than you do.
- [09:50] Dmitriy Grankin:  like a purchase acquisition and there is no way to go but it will be as i see the market at least two three four years So, how does it work? Do you have enterprise B2B subscription model or how does it work?  For now, it's just free of charge. Just do what you want.  And so do you count the number of users you have so far?  Of course, yes, yes, yes.  So how is it looking?  It's like 300, 400 users.  about those who are using at least three meetings a week, there is about 50 of them.  And the group retention, they are marked Weeks and months during like some other 500 meetings they have recorded with the crew.  And do you send all these recordings and all the data to just to...  The transcription model, which is a waste tax, is a local model on my own.

## Meeting 25 - November 25, 2024 15:20
Topic: Рефакторинг и самокритика

- [12:20] Dmitriy Grankin:  сбираться, оно мне нравится, очень сильно мне нравится.  Сюжет, думаю, идти мне рефакторить или не идти рефакторить.  пойти за рефакторе потому что это просто какое-то просто издевательство сама  самобичевание вопросам хотя работала конечно все вроде  В любом случае, если ты чувствуешь...  Это дело, наверное, много причин и...

## Meeting 24 - December 05, 2024 11:02
Topic: Product-Led Growth

- [14:00] Dmitry Grankin:  Это механизм, но они используют широко интеграцию с календарем, Ты получаешь имейлы всех пользователей на звонках своих пользователей, рассылаешь им фоллоуапы.  вот это вот это наверное еще круче интересно наверное вот но здесь надо искать ценность для того чтобы получить эту интеграцию с календарем  Да, есть такое. Сложно. Не знаю.  Я помню, мы пробовали много с инфлюенсерами, а там гипотеза.  Там нужно очень много... Там нужно, чтобы продукт был невероятно привлекательный со всех сторон, именно как первый шаг. Остальные шаги неважны.  То есть там чаще даже делают какой-то очень внешне X-овый привлекательный продукт с классными превьюшками, но он может на первом же этапе ломаться и не работать дальше.  Но так...  Проверяют, идет ли трафик или нет.  Но это вообще с другой стороны.  Мне кажется, что не стоит туда идти.  Там свои проблемы.  Там продукт разрабатывается так, что сначала делается фасад.  а потом уже fake it until I make it, через годик будет продукт.  Мне кажется,  у тебя прям точно можно цеплять именно тем, что это готовый продукт, он работает.  Знаешь,  с инфлюенсерами у нас как раз есть хорошая история и там нанимал девочку бразильянку искал инфлюенсеров одного микро инфлюенсер у него там 3000 подписчиков он записал один видосик и оттуда каждый день идут пользователи уже на протяжении двух месяцев Нет проблем с трафиком с инфлюенсерами.  Есть проблемы с целевым трафиком с инфлюенсерами.  Но они пользуются.  А, пользуются.  Они пользуются, каждый день потом пользуются.  Слушай, если идет трафик с инфлюенсеров, тогда реально, может быть, идти в сторону инфлюенсеров.  Да, и потом видишь, у меня какие шансы были обратиться к той или иной.  Они по-английски не говорят, он по-английски не говорит. Я не могу с ним даже переписываться, он так по-английски пишет, что его понять невозможно, что он хочет сказать.  Вот, то есть прямо история, которая прямо кажется, что ее надо масштабировать.  Много появляется платформ для работы с инфлюенсерами.  Там, наверное, вопрос скорее цены, чтобы сходил с юнит-экономикой, что возврат инвестиций будет.  Да.  но там я 200 евро потратил это он кстати бесплатно сделал он получил год бесплатного использует последний Вот, и да, наверное, вот это да, наверное, нужно делать в первую очередь, ну вот, а разработку, да, вот вести, исходя из того, а что привлечет больше пользователей, вот.  давайте вопрос я делаю разработка что даст сетевой эффект из этого из того что делаешь ну вот и чисто ориентироваться на то что что обратку тебя отвечают пользователи. Больше ничего больше не надо делать.  И даже с инвесторами не разговаривать.  Сами придут.  Да, хороший план.  Значит, как план?  Вообще я вижу много разных способов того, как люди привлекают трафик к себе, но они все контентные, к сожалению.  То есть я даже не могу ничего посоветовать хорошего.  Ну, то есть, как пример.  Есть ребята, которые делают какой-то контент в Twitter, в Blue Sky.  в LinkedIn, где рассказывают о том, как они показывают, как они что-то разработали, и там чисто на красоту. То есть они просто показывают какую-то новую фичу, скорее всего, даже не фичу, какую-то мелочь, которую они добавили.  И просто делают это каждый То есть, очевидно, они нанимают какого-то разработчика, который занимается только какой-то привлекательной частью продукта и шарит просто какие-то мелочи.  Есть ребята, которые покупают...  маленькие продукты.  Вот это, кстати, очень любопытная гипотеза, но есть ребята, которые покупают у инди-хакеров продукты, которые Пишут, что они их сделали, как бы собирая портфель этих микропродуктов.  На самом деле все эти микропродукты просто это как контентный маркетинг, и они пиарят его как личный бренд и его продукт.  выдвигая его вперед и любопытно что таких становится все больше и больше тоже как вариант но это мне кажется смежно с тем что ты рассказал про сел тулы Да, про микросарс, про селтулы.  Мне кажется, Саша, в целом, что сейчас такой органический трафик из социальных сетей становится чуть более обширным каналом трафика, чем SEO.  Но это точно зависит от аудитории.  В общем, да, нужно смотреть.  Ну, это же неизолированные каналы.  У тебя есть SEO-трафик, просто какая-то потребность, если ты ее выявляешь с помощью SEO.  Uh, статистике поисковых машин но значит иди и там на ютюбе сделай интеграции какие-то или просто накидай туда контента получишь там трафик и конвертирую его или как-то по-другому вообще наверное с этого вообще надо любой проект начинать именно так не от своих фантазии, глобальных представлений о том, куда мир идет и что ему надо.  А именно вот с этого и дальше уже итерироваться.  Ну вот, начали так, как начали, а теперь все от обратного будет.  По крайней мере, все равно, видишь, здесь кажется, что зачатки рынка существуют, потребности есть.  Есть конкуренты, есть, как автобус едет, в нем 5 пассажиров, легко, свободно, куда-то едет и куда-нибудь приедет.  Но еще не скоро, еще 2-3-4 года есть, пока произойдет какое-то насыщение, какая-то...  консолидации по динамике по крайней мере выглядит так может кто-то сейчас придет и всех съезд моментально курсу от пришел и съел всякие там все экстеншены для вс кода пользоваться теперь нормальной интеграции  Да, но здесь, наверное, только если какой-нибудь антропик или OpenAI решит сделать чуть-чуть больше интеграции с компом и решит сделать отдельное приложение.  Положение, которое будет и записывать звонки, и еще и внутренний научный менеджмент, мне кажется, что это слишком… это не универсальный юзкейс.
- [22:00] Alex Shevliakov:  Я не понимаю, зачем вам туда идти.  Но knowledge management звучит как универсальный use case, и дальше у тебя просто каналы данных, да, каналы получения текстов, да, у тебя.  есть переписка есть звонки есть сайты есть заметки все все наверное куда-то туда придти какой-то момент потому что и мне понятно и пользователи все говорят да клёво а почему только звонки а можно я туда логичный абсолютно очевидный вопрос и очевидно ничего не останавливает ему У меня есть в Postgres колонка транскрипт, а ее можно переназвать по-другому.  назвать или даже не менять ее название, класть туда что угодно вообще, любой текст.  Да, применения очень много.  Если сделать, что это просто подключаемое, вернее, подключаемое, База данных, куда собираются разные типы информации, и потом по ней можно сетчить или даже сделать какого-то чат-бота, с которым можно чатиться, не знаю, в инфраструктуре компании.  Ты, Глеб, очень привлекательный.  Да.  Вот.  Так что я ищу какого взаимодействия.  У меня накопился сейчас опыт работы с людьми.  И с разработчиками, да, у меня был, значит, Scrum Master, Backend, Frontend, Q-инженер, у нас были, значит, утренние, значит, делики, недельные.  ретро планирования а потом это все не стало и стал получаться в 10 раз быстрее так и есть и и затрат и и все полетело потому что я бы кента фронт-энд делаю одновременно без без ретро и без сен-сен сенков и обмене почему И мой запрос, понятно, я хочу себя масштабировать, а желательно не себя масштабировать, а чтобы, естественно, человек, с которым я работаю.  естественно были сильные стороны которых у меня нет но я себя вижу так это просто чисто онлайн вот мы сидим И мы вот эту скульптуру делаем вдвоем.  Мы прибыли, у нас есть информационный поток от пользователей, у нас есть информационный, у нас есть собственное представление.  прекрасном у нас есть результат наших экспериментов которые прямо сейчас и возникает но это вот такое как бы может сказать что-то вроде парного программирования это когда происходит риэлта и как бы мультиплицируется курсором конкретный инструмент например я вчера вот этот вот механизм шеринга и бэкэнд и фронтэнд, я вчера сделал вечером, я думал, что мне такое сделать, чтобы мне было, как бы у меня что-то вот уже с Спать хочется, внимания что-то никакого нет.  Надо что-то такое сделать изолированное.  Сделал кэшеринг.  То есть, если бы мне сказали, что я так два месяца назад, что это будет возможно, я бы сказал, ребята, я...  представляю себе одновременно и фронт и бэк потому что я сказал и мне лень у меня нет ресурса поэтому я напишу туда сделаем не sharing и напишу сюда здесь мишени а потом я ему лобби рифлин там раскидаю чтобы они посмотрели что не так ну грубо говоря конечно важнее естественно но все равно во многом как-то этого вот вот вот вот такой слог и понимаешь зато этот слог и превращается в супер быстро и я супер быстро начну получать обратку а потом когда у меня bottleneck я его увижу, вот тогда значит мы будем его расшивать.  Звучит логично.  А что, было бы прикольно вот в каком-то В каком формате поработать, если тебе интересно, дай мне знать, вот у меня всегда для этого есть запрос.  Хорошо. У меня пока напряженно со временем, но да, конечно.  Буду знать, буду думать об этом.  Да, ну вот, и дизайн точно так же, да, все прямо вот в одном, вот в одном потоке все.  да понимаю это как раньше люди писали письма вот у тебя письмо идет долго поэтому я его пишу долго тщательно переписываю три месяца на и она идет люди смачно недели его читают всей семьей это этот вид вот это вот та работа который сейчас занимаются я написал что-то такое с ошибками по марками ты прочитала я потом наряжал иди ты и зайдите тебе  Да, есть такое.  Вот разница того, что внезапно, буквально несколько недель назад технологически изменилась.  Люди, которые этого не понимают, мне реально их искренне жаль, потому что я не нашел ни одного человека, который бы мог эту идею донести, чтобы он ее прочувствовал.  разделил мне все почему это не работает это работает просто отработает это работает когда у ...в плане одновременно делать продукт.  Ну, то есть это не работает, к сожалению, с подрядчиками.  Это не работает с наймом.  Это работает вот...  Мы с другом...  вот тут уже с другом партнером бизнесом делаем штуку мы выделили время мы все время собираемся в одно и то же время физически и садимся Открываем ноутбуки и делаем так.  Он занимается в основном бэкэндовой частью, бизнесом и рекламой.
- [29:00] Alex Shevliakov:  Я там занимаюсь приложением, фронтовым контентом.  мы все время делаем все одновременно. И ни у кого нет никаких, ну то есть иногда что-то планируем, что-то спрашиваем, но никаких дейликов, никаких синков, потому что каждый из нас знает, зачем он это делает, почему ему хочется это делать. И все, как бы проблемы решены.  Так что, конечно, я тебя очень понимаю.  И онлайн это тоже так же отлично работает.  Так что...  Ты знаешь, вот я встречался недавно со знакомым, он открыл венчурную студию и он мне то же самое рассказывал.  ...о том, что вот они сняли офис, и там их семь человек, они приходят и все вместе делают, и они там не как основатели и...  нанятые люди, а у них как бы все на шару.  И все это очень хорошо работает, и они быстро двигаются, и у них работает так, что каждый день они себе задают вопрос, а что я сегодня сделал, что я сегодня добавил.  И нужно каждый день себе отвечать на вопрос, что я сегодня добавил и почему я это сделал.  И это сразу работает.  Да, причем не что я сегодня собираюсь добавить.  потому что ты можешь ты не знаешь скульптор вот этот вот что получилось а чё за чуваки  я не знаю до конца что именно не делают его зовут уильям каблер и они открыли И они делают много разных апов, но Титалия будет спрашивать потом, потому что мы просто встречались в кофе попить.  Посмотрим. Он просто в телеге не отвечает. Я с ним вообще через Dribbble познакомился.  Вот я буду ещё раз на следующей неделе пить кофе.  поближе познакомлюсь.  А еще здесь есть  инди-хакерская комьюнити собирается.  Вот. Так что...  Я не знаю, ты знаешь про такое?  Называется «Инди-бирс».  Слышал, наверное, да, наверное, в этих, да, в Митапе, да, может быть, но не в 3D.  через twitter обычно собирают там у них раньше собирал бернета который делал тайный хост в общем довольно тоже интересные чуваки я думаю тебе будет Будет интересно пообщаться, потому что у них в целом такие же мысли, но они в основном те, с которыми я общался, они обысадоры того, что нужно одному все делать, и тогда все будет хорошо работать.  просто в какой-то момент все равно надо масштабироваться да все равно куда тебя инструменты масштабируют очень сильно но если тебе надо быстрее а те в какой-то момент все равно надо быстрее вот ну и потом вместе все равно веселее вместе точно силе я поэтому вот это  Мы с другом начали делать проект, потому что есть такой фактор, что когда у одного там что-то застревает, у второго двигается, и все равно шерите этот прогресс.  Есть такой важный стоп?  Да, там статистика лучше получается, более сглаженная.  Ещё из суперактивных таких хэндихакеров я знаю, Я знаю Марка, он прям все время хочет со всеми общаться.  Давайте я скину тележку его, если вдруг ты его не знаешь, то...  скажи просто, что от меня, и что тоже.  Что селл-пренер, он будет очень рад пообщаться.  Клево.  У него все время тонна идей, он каждый две недели мне говорит о том, что слушай, у меня тут такая идея, все надо делать.  окей ну ладно давай тогда чего давай тогда до связи на связи да пиши какие будут мысли да ты  Да, по поводу сайта я хотел сказать о том, что мне немножко сложно по сайту понять, какая основная идея, ценность потому что там классно описаны продукты но они все немножко про разные штуки то есть как будто решил несколько несколько вариантов ценностей разместить ...постить на сайте и посмотреть, какая... какая пользователь зайдет.  Это классная штука, но я очень рекомендую для этого делать отдельные странички, потому что это и для SEO-шки будет лучше, у тебя разные запросы будут вести на разные...  страничке я думаю что с этим вино ли мы это хорошо ты знаешь истории этого логин  пейдж это по большому счету я написал фронтента Зачем мне кормить эту дурацкую тильду? Все равно надо сделать нормальную страничку.  Тем более, что это вообще элементарно.  задача и как бы потом ну да вроде страница есть руки и пошли дальше то есть good enough  не более того да я просто о том что у тебя же трафик на сайт и это будет такой важный ступень между пользователем и конверсией в пользователя.  Однозначно, конечно.  У меня там даже нет Google Аналитики еще.  То есть вот это вот...  ну то есть просто тупо не хватает времени поэтому как бы тоже тоже  масштабироваться все равно хоть хотелось бы понимаю  Вот такие дела.  Ну ладно.  Хорошо, тогда...  Да, если будет  желание, возможность  покопаться в том, что я сделал,  и сделать обратку, будет круто.  Да, хорошо. Давай я посмотрю, как будет со временем.  Я буду помнить, что как времечко появится, я сразу тебе напишу.  Да. Спасибо тебе.  Да, тебе тоже спасибо. Давай, пока-пока.

## Meeting 23 - November 25, 2024 10:13
Topic: Тестовая запись и распознавание спикеров

- [00:00] :  Делаю тестовую запись для того, чтобы посмотреть, как будут спикеры распознаваться, записываться.

## Meeting 22 - December 04, 2024 15:32
Topic: Product Development and Features

- [02:42] This chunk is part of a conversation between Jasper Vanuytrecht and Dmitry Grankin, where they discuss the functionality of a transcription tool that supports multiple languages, including Dutch. Jasper shares his positive experience using the tool for training sessions and workshops, highlighting its effectiveness in transcribing Dutch. The discussion also touches on the tool's features, such as real-time language switching and meeting summaries.
Jasper Vanuytrecht:  good job so you have created a fax product yes did you have a chance to try no or not yet as i say i'm using grain i'm so far happy because it also transcripts other languages like dutch language I do a lot of training sessions, I give workshops in Dutch, so it's really useful for me.  Does it work well in Dutch?  yes yes yes i was surprised because i tried some other applications like fatom i think and also uh but it was not that good okay uh can you say something in dutch now  so we will see how wexel cope with it okay  Ik ben een consultant, ik werk graag met LinkedIn en ik woon in Spanje sinds 2013.  not not that different not that different from uh from english yeah can you see it  I'm not seeing, let me check.  Yeah, I'm sharing the screen now.  Yeah, but it's like, oh, everything is like, I don't know, I see a lot of windows.  haha you lost me oh yeah yeah that's that's because I'm sharing my screen which is the screen we're having with you right now that's the issue But what I'm showing you is my Chrome extension, which is delivering these things real time.  that's probably pretty good yeah yeah so here is what you've been saying so it's it's switching automatically um between languages and does some highlights so i can you know i can go back and see you mention this Graeme, I just give it a click and Jasper mentioned Graeme during the call.  It's a tool that describes meeting and supports multiple languages, including Dutch.
- [05:42] Dmitry Grankin:  And after the meeting we got a dashboard like like this one which gives a summary and it gives a chat gpt style chat which can generate stuff based on a specific meeting or basically all the meetings as well.  What happened last week?  Tactic AI extension.  Tactic?  Yeah, I've been playing around with this one because it's a pretty similar concept.  uh-huh but it's putting this in Russian for some reason probably because I've got a lot of context in Russian no yeah Yeah, so the idea is basically to create knowledge that's accessible in in charge of the key manner.  Okay.  And it's it's success.  both real time during the call and after the call yeah so it's it's more like  like a chat interface, no, instead of static reports, you can.  The idea is that like chat is, is, is kind of central thing.  But also the meetings and the filtering is that I can filter a subset of meetings and chart over this specific subset.  Like in a future release, it will be like you could just filter out your clients or team members.  numbers and chat over these ones all right kind of free trial  for the product yeah yeah you can you can go and uh  and use it full like a full version of it for free now because like payment is not implement  the theme is not implemented yet this feature is not yet delivered well it's a good way to to get traction, to get some users.  Yeah.  To see how they perform.  Like it's in the pipeline,  but for now it looks like it is a product that's ready to to get some sales because uh there is a stable user base like with some people are using the product since early days for 500 meetings a few hundred victims there's a few users like this and we got a stable influx of users every day like a few like two to five users daily with no marketing whatsoever yeah yeah I think it's a solution for companies that have  of meetings and several people in the company, you know.  So do you have your VC backed or was it the previous group?  yeah it's VC backed like I've raised some angel investment mm-hmm Now this year Okay, and I think we'll be able probably be able to to go this way up to serious a maybe all right you said you have clients like new clients they go from your existing  network community like all warm no not really for now these are the clients that are coming with  empty utms which which basically means it's a direct uh customer directly referenced by someone i oh yeah yeah referenced right yes i don't i don't see the reference itself because oh yeah yeah referenced right yes i don't i don't see the reference itself because  but you're not doing like uh outreach to businesses to offer the trial or to talk to them about yeah how they are doing their meetings currently you know they're sharing

## Meeting 21 - December 02, 2024 17:27
Topic: Meeting Page Features

- [02:36] Dmitriy Grankin:  I click a meeting with tops opens a meeting page and we have pretty much a lot pretty a lot of things this way I have like I open now I'm meeting note yeah looks like it's not Jestem w kosmicznej dyskusji, mam Cię.  Okej.
- [02:06] Dmitriy Grankin:  On the dashboard, we have list of meetings, we have sidebar, so it's meeting centered.
- [01:43] Dmitriy Grankin:  tactic it wants me to has requested additional okay it requested to change all your data Oh, that's a lot.
- [30:36] Dmitriy Grankin:  label conference long create new label labels label yeah labeling is a good idea okay basically that's it okay let's move forward action okay because me i told so which i just don't get it why what's that maybe it's um kann da gast und prompt automatically insert um ok, my prompt Herzlich Willkommen bei Tchau.
- [22:02] Dmitriy Grankin:  archive. I have these buttons Building, okay, and I got this.  okay and they want feedback they won't let me edit everyone action items for everyone we got the section items they are pretty huge Shirt with everyone in spaces, no recording, recording link.

## Meeting 20 - November 24, 2024 22:09
Topic: Weather Reference

- [01:09] Dmitriy Grankin:  Tempo de chuva.  Потому что с этим у меня там было в морочке.  Почему я ничего не вижу?  Вижу.  Эти нокаиды неправильно указывают.  Это может быть.
- [05:09] Dmitriy Grankin:  تو ہوتا ہوں Сделаем, посмотрим, что будет.

## Meeting 19 - December 11, 2024 18:31
Topic: Setting Up Meetings

- [24:22] Dmitriy Grankin:  which is already a lot of information from this meeting possibly if we if we're talking about something that we for sure would understand if there is anything about this at all if we are in the same share share at least some common ground at all if we do share then it's a win if we don't it's already it's also an information a lot of information okay right so we should  really start i should really start reaching out to the people regarding the Project management.  We don't know, because it's just a piece of text that we're putting into the system.  system uh how to set up trello for product management you can you you can put whatever request like how to uh how to make a paper ship right and it will work the same way okay

## Meeting 18 - December 05, 2024 10:02
Topic: Conversation Recording

- [02:19] Oleg Ageychev:  Все, что я знаю, в разговорах я проговариваю, оно фиксируется.
- [05:19] Dmitry Grankin:  эти вот элемент вот они как бы они проекта индекс по которому
- [05:12] Oleg Ageychev:  все беседы или вообще всего что он знает всего что он насобирал потому что вот
- [02:00] Dmitry Grankin:  Вот, потому что это больше, и она не только больше, она еще и сама по себе имеет большую ценность для пользователя.
- [01:54] Dmitry Grankin:  начинается, когда мы начинаем собирать и давать пользователям доступ к их личной информации.
- [17:32] Dmitry Grankin:  так я тебе дам это действительно такая существующая уже ниша вот это не старта вакууме это прям такой потребность мы с конкурентами совершенно определенно нащупали вот заключается она в том чтобы что-то получить новое, чего раньше не было, и не потерять эту информацию.  То есть люди хотят сохранить информацию для себя.
- [27:10] Dmitry Grankin:  Вот. А что у меня происходило? У меня же еще были, у меня был, значит… Back-end, Front-end, Scrum-master, QA-Engineer.  Это те, кем ты обложился, что?  быстрее сделать и опять что происходит два раза в день созвоны денег вот потом раз в неделю планировать следующей неделе по факту я потихонечку со всеми попрощался появился курсор это Это продукт, ну, я по факту, вот я всех уволил, я один, и я гораздо быстрее бегу.  Я потратил на разработку с ними.  40000 евро а теперь я один сделал больше и быстрее за за последний месяц я понимаю  Боли, поскольку я же тоже из тех, поэтому у меня много друзей, которые сейчас работают в CTO и так далее, и это то, что ты говоришь, это общая боль. В смысле, что вроде как вот...  Мне самому, особенно когда я умный и работоспособный и замотивированный, я лучше это сделаю и напишу.  Может быть, я потрачу в полтора раза больше времени или в два.  Но зато все будет сделано качественно, а так я много разговариваю, а результат все дальше и дальше.
- [14:51] Oleg Ageychev:  А мы делаем такие чуть более изящные штуки но у них есть дополнительные которые на следующем уже уровне бизнеса начинают проявляться ну вот типа знания решение репутация образ некий как вот это вот все выглядит да там видишь потому что все таки тема же 5 она там во первых есть очевидные понятно что есть очень сильные промоутеры вот точно также уже появляются те кто начинают сопротивляться Сейчас захватят и так далее. Поэтому там появляется и тревожность, и все такое, за этим всем надо работать. Как-то. Я в данном случае не только тебя имею в виду, а вообще целый сектор.  Сейчас как бы намечаются такие проблемы, и законодательство, и так далее, там совсем с этим мало чего.
- [01:16] Dmitry Grankin:  Вот, интернет, он из себя представляет процентов 10, наверное, всех данных, которые есть, потому что большая часть данных, она застревает.
- [38:05] :  хорошо дим на этом давай наверно закончим на сегодня ссылки на конкурентов ну из  Вексу я сейчас тоже буду крутить, и что-то с ней пока поваляется.  Окей, договорились. Давай.  Давай, на связи.  Пока.
- [01:03] Dmitry Grankin:  Это общедоступные данные, это условно некий интернет.
- [05:39] Oleg Ageychev:  дальше работает выглядит очень симпатично а что есть еще из Таню Сухову, с теми же людьми, с которыми, в общем, я. А ты, кстати, да, ты сейчас разишь деньги, правильно я понимаю?
- [04:39] Dmitry Grankin:  что мы разобрали разговор на знание на единицы знаний то что я там человеку рассказ Вот, пояснялка, вот, короткое summary разговора, название, вот, поэтому разговоры я могу вытащить, какие-то подробности.
- [07:43] Oleg Ageychev:  все там чему я ее обучил про свой контекст про то как я работаю про то там вот я прямо вообще даже без разговоров с людьми прямо с ней так сел и поговорил много раз плотно хочу себе такого вот ассистента воспитать вот значит наговаривая ему разные там штуки и эти данные все все что я наговариваю для меня она прямо сенситив потому что это там в том числе этом про авторские права и так далее юридические там вопрос вот я Это все хранится у тебя секьюрно.  И я имею в виду, что у тебя же ядро одно.
- [00:35] Dmitry Grankin:  версию общаюсь вот показываю что у меня есть еще резонанс и понятно Ну, в общем, векция – это ассистент для Google Meet, который собирает знания, дает им доступ.
- [05:56] Dmitry Grankin:  Даже, ну, то есть у меня что-то есть, я хочу этим поделиться в прямом смысле, то есть я хочу об этом рассказать, я хочу получить пользователей.  я хочу привлечь компании хочу получить фидбэк вот то есть у меня пока потребности финансирования нет как такое надеюсь и не будет монетизировать то в итоге будет

## Meeting 17 - November 25, 2024 20:50
Topic: Greeting

- [00:00] Dmitriy Grankin:  Да, привет, привет, это опять ты.
- [02:48] :  Теперь мы её, например, закрываем.
- [01:23] Dmitriy Grankin:  Короче, в очередной раз умотался, ушатался.

## Meeting 16 - December 13, 2024 11:30
Topic: Структурирование знаний

- [05:23] Viktoria Kharlamova:  если я правильно понимаю, то то, куда вы движетесь, это какой-то умный хаб всех знаний внутри компании, всех проектов. Вот. Ну, соответственно, вы просто будете больше структурировать.
- [06:09] Dmitry Grankin:  Мы движемся от идеи митинг-ноутейкера, базовая потребность, которая закрывает митинг-ноутейкер, людей это том что я записала она у меня там есть я потом как-то на этом увернуться могу вернуться потом они по сути дела практически и не возвращаются вот мы да мы хотим дальше есть какая концепция есть Там есть продукты типа ChatGPT, Clot AI, это продукты, которые...  которые дают доступ к общей информации, к информации из интернета, которая общедоступна.  Общедоступная информация – это круто, и мы ее можем генерализовать, создавать какие-то новые данные, но на самом деле gated data, какие-то Личные данные. Они, во-первых, большая часть интернета представляет, они больше. Во-вторых, они ценнее.  Соответственно, дать доступ пользователю к этим данным, это то, наверное, в какую сторону движется бизнес.  продукты которые ну тоже те же самые но такера они по-любому движется в какую-то в эту сторону даже если они этого не знают вот мы хотим прямо про артикулировать и Прямо заявить о продукт, который это делает.  Ну и вот частично он эту задачу сейчас решает.  Технологически совершенно понятно, как это делать.
- [04:20] Dmitry Grankin:  Chrome Extension, если...  Про анбординг говорить. Установили там три кнопки и попали в дашборд. Дашборд или звонок.  по тому что у нас с пользователями происходит со статистикой давай прям покажу чтобы не рассказывать что это такое я лучше в таких сумму статистику ты сама делала вывод насколько это смотри пока ищешь быстро задам еще вопрос то есть концептуально
- [01:16] Dmitry Grankin:  совокупности, которые выглядят таким образом. У нас сюда вываливаются встречи, короткое summary, hit points of interest и у нас упорно чат у нас чат по встрече чат по спикеру можно взять какого-то отдельного спикера и про него все.  Прикольно. В общем, все структурнее стало, получается.  Да. Вот. Есть глобальный чат.  Да, можно...
- [11:38] Viktoria Kharlamova:  Я правильно понимаю, что, по сути, основная задача, которую ты сейчас ставишь, скорее всего, это две истории.  больше входящий трафик, и те, кто устанавливали, чтобы они оставались с продуктом?  Я думаю, с точки зрения ретеншена, я вижу, что он есть.  Ну, пока непонятно какой.
- [02:34] Viktoria Kharlamova:  Правильно я понимаю, что в том числе Meeting Notes, они будут подтягиваться туда или пока что нет?  Meeting Notes, куда туда будут подтягиваться?  Но имеется в виду, смотри,  То есть то, что я помню, каким образом выглядел продукт, это были в основном meeting notes. Ну, то есть я ассистент, который записывал у тебя то, о чем разговаривали на встрече.  Транскрипты, да, вот, вот.  Да, да, да.  Транскрипт, который вычищенный и с хайлайтами.  Да, и здесь они есть, да, вот наш транскрипт вот здесь.  И получается, что помимо этого, то есть этот транскрипт будет подтягиваться сюда и автоматически добавляться, то есть либо можно будет быстро добавить что-то в определенные проекты, я правильно понимаю? Ну, именно проекты.  Да, он автоматически уже здесь, наш транскрипт уже здесь, пока мы говорим. Все здесь. Дальше концептуально.  Мы этот output или какой-то другой текст добавляем сюда же в проекты, какие-то отчеты, которые тоже...  накапливаются и становятся доступны для чата.
- [24:23] Viktoria Kharlamova:  Смотри, у меня вопрос по поводу того, я сейчас в финале планы на Ку-1.  На Q2 у меня, ну, первый квартал и второй квартал именно по клиентам и по загруженности.  Вот, у меня осталось два местечка, и на следующей неделе еще примерно пять звонков.  Вот, и, соответственно, я хотела бы понять, сколько тебе времени нужно подумать, принять решение и с точки зрения начала, если бы ты хотела начинать, когда бы это было?  Слушай, ну ставка такая для меня драконовская довольно-таки.  Да, чуть попозже.  Ага, окей, окей, я поняла.  Ну смотри, тогда в любом случае, я думаю, что...  Когда, не знаю, будет трафик, вы протестите, и будет другая стадия, может быть, внедрите самостоятельную монетизацию и захотите скейлиться, тогда уже будет смысл вместе поработать.  Да.  Окей. Да, я думаю, так, наверное, и будет.  Супер. Тогда здорово, что написал.  Классно, что вы движетесь в интересном направлении.  У тебя такие зонты.  какие в мете проходят в зуме или как слушай либо в мете либо в зуме ну то  есть 50 на 50 примерно используешь какие-то такие инструменты я использовала tldr по-моему назывался вот мне не очень нравится я вообще поняла что они мне не очень нужны я делаю самозметки вот плюс мне не очень нравится отсутствие но короче не все инструменты следует правилам секьюрности ну то есть я считаю что когда ты что-то записываешь  важно чтобы человек об этом знал что тебя смешает сообщить об этом  Ну, я думаю, что это должно быть проактивно со стороны инструмента, скорее, вот так вот происходить.  Все так или иначе каким-то образом сообщают, но, например, да.  например это делаю через чат но никто не видит в основном нам как раз бот приходит прямо на звонок сидит на звонке он прекрасно видно.  Ну да, да, да, да, часто вот формат такой, он в принципе окей.  Это людей, кому как, как выяснилось от обратной связи от пользователей, главное преимущество.  моего продукта это то что бот не висит на звонке не то что люди знают или не знают о том что он записывает о том что вот такая вот 3 3 человек сейчас третьим  аватар вот и 2 мульти язычность то что можно в любой момент приключиться на любой другой язык  Угу. Да, да, да, да, это точно.  Good. Тогда рада была поболтать. Вот, надеюсь, что...  получится заимплементировать классную монетизацию, сделать первые результаты,  начать растить продукты, потом, если понадобится помощь, обязательно пиши.  Давай, спасибо тебе большое.  Давай, хорошего дня.
- [03:50] Dmitry Grankin:  Вот, какие-то внешние источники данных.  То есть это такой, по сути, knowledge hub проектовых.  где вся информация хранится и может быть структурирована.  Здесь какие-то визуалки.  Да, этот сайт я посмотрела, то есть ваше главное, как поменялось.  Да.

## Meeting 15 - November 25, 2024 14:56
Topic: Pandas и SQL

- [02:48] :  берем pandas и берем select start from speeches order by  Ох, слава тебе, Господи, работает, а? Работает?
- [01:36] :  И пересобираемся.  Теперь опять нужно сделать...

## Meeting 14 - December 12, 2024 16:00
Topic: Personal Note-taking Habits

- [06:02] Dmitry Grankin:  So you have some experience using the products, but, understand there is no click with you so you don't find it useful enough even to to subscribe to a premium version of such a product I don't know if that's the case.  I think.  Yeah, that's not a go all out Phil answer, and then I'll give my opinion.  Yeah, no, personally, I just have always taken notes Like while I'm doing meetings.  So it's just like a habit that I, I already have formed and it's like, okay, uh, this, this thing does it for me, but I kind of understand better when I'm the one taking the notes.  Um, so I haven't like gotten.  that gap, I guess, but I'm definitely open to it.  And it's super helpful right on this.  I, yeah.  And my personal opinion is that I, you know I don't have great habits with respect making notes actually, but I, so I don't like, I'm not a big, I don't like sending it to customers or clients. You know, I like keeping it for myself. That's one thing.  And the second thing is like, I guess I just always kind of view it as like, like I know it's 100% like the value, I see it, but I just, I feel like it always ends up devolving into me just seeing it as like another thing that I have to do.  You know what I mean?  Like, and like, I feel like, like, I don't know, but, but that being said, what I, the way I use it is I use Otter.  That's the one that I was looking for.  I, I just have it.  transcribe the meetings and like i you know put it through like a gpt and like summarize it and make it further and then i store it away for myself for like most calls that i do yeah pretty much everyone but phil because i talk to phil a lot  Yeah, and what is the result you get from these GPTs, or basically what have you been prompting for?
- [07:51] AI Marketing Directory:  just to like shorten it and summarize it like nothing too complicated yeah yeah like like like just so because um that's it yeah okay and uh the final destination is that do you do you  actually stored this information somewhere where you at least able to get back later or yeah mm-hmm keep it in a Google Drive folder okay you put it as a Google document and do you find yourself getting back to this archive in any point in time?  I do in
- [08:42] AI Marketing Directory:  literally only one specific i started doing it for one reason because our third partner um is an older guy and he uh he you know sometimes when i like talk to him about calls that i have a little summary so I literally you know if he asked me I'll pull it up but but that's pretty much the only use I found for it so far you've been reporting with was this  so you've been creating reports about your communities with with the other partner  no no so like i summarize my i don't use it for like you know sometimes You know, like I, I do it for, you know, I summarize like a call that I had, I'll put it, you know, in like a Google drive document and like, you know, if, if my, if the other partner asks us about it, I'll pull it up on my phone.  I'll just read it you know okay I don't do anything else with it after that yeah  basically you want to recall at least some general idea what was going on
- [04:43] Dmitry Grankin:  feel of flexibility and willingness to to no more yeah and so yeah to collaborate so okay um i can i can share my progress with you know absolutely So Vexa is a meeting note taker, basically, which is probably you've been encountering

## Meeting 13 - December 03, 2024 14:01
Topic: Обновление расширений

- [01:00] Dmitriy Grankin:  но вообще даст наверное сейчас лучше всего это сделать в магазине но там и ссылочка есть Как обновить?  Вот там deactivate, activate, наверное, только.  Да, вижу. Удалитесь.  Окей, допустим, remove.  Говорит, типа, из-за того, что abuse.  Нет, из-за того, что вы, идиоты, не сделали кнопку обновить.  Да, это суперстрэнч.  Так, мне теперь надо его как-то запинить.  Все, окей, запинить.  Но оно обновляется, если просто полностью закрыть браузер, но так не все.  Да, 10.10.02, но картиночка все еще белая, я не знаю ответа.  Ты это менял, не менял?  Картиночка, а что за картиночка?  А, картиночка, я понял тебя.  Картиночку нет, картиночку я не менял.  Но Александр уже сам включается.  на этом на фоне ты же ее не наблюдаешь эту картиночку ну понятно что надо сделать но  они просто мне просто удобно знаешь пинить вот это счет screen скину я вроде скину да  Да, да, я понял, но он тебе этот пин вроде ничего не даст, потому что она все равно...  Но помнишь, мы обсуждали, что не очень очевидно, как попасть в дашборд, например, да?  И было бы неплохо это сделать через нажатие вот на эту картиночку.  Ну, то есть, во всяком случае, для меня это самый понятный UX был бы.  Я тебя понял.  Так, окей.  Да, логично.  Еще какие новости?  Какие новости?  Да вот, это я прям сидел, вставал.  Запотел?  Несколько дней, несколько недель.  Вот.  Ну, там, это...  лендинг обновился вместе со всем остальным. Там дешборд. Это, в общем, Я иду в сторону знаний. Мы говорим, мы делимся, мы все это превращаем в знания, доступные в чате.  Угу, супер, да, вижу свои созвонщики, у меня уже штук 15 накопилось.  Угу. Окей.  Есть там эти... Summaries, да?  Так, допустим, стартапы внедрения технологий в Газпром. Да, прикольно, красивая самая реальная. Так, так, так, сейчас обсуждение стартаповых условий.  Слово, это дата, филипп Сильверсов, 3D-модель.  Окей.  Мне, знаешь, еще нравилось, в предыдущей версии, сейчас пока сход не вижу, готовые кнопочки, там, типа...  Action Items и какие-то еще.  Вот я Action Items чаще всего использовал. Это осталось, не осталось?  Да, сделаю, сделаю.  Окей, хорошо.  Сделаю перед пользователем, как школьник записываю.  Нет, это правильно, это круто.  Да, да, да.  Самое главное, что я теперь это могу делать сам и, типа, это сказать.  Скорее всего, ну, типа, полчаса.  А, ты говорил, что там, типа, это фронт тебе помогал, да.  Да, теперь мне не нужен никакой фронт, я все в 10 раз быстрее делаю.  Ну, я, кстати, примерно к такой же штуке пришел, я тоже сам начал немножко фронт подпиливать у себя на проекте, и это действительно бустит. Вообще, фулл стеки машины, конечно.  ты ходишь забавный вопрос вообще не забавный потому что это делаю совершенно по-другому  совсем не так как два месяца назад я больше не пишу код никакой я его читаю в лучшем случае В лучшем случае читаю.  Это круто. Я, знаешь,  какую штукенцию завязал?  Для парочки таких типа условно-проектиков допустим, нам нужно...  Нужно, вот у меня партнеры, у нас же открытая экспортная компания вруговая, и я там жду, когда сделочки поедут, чтобы сделать токенизацию цепочек поставок, посмотреть, где с IT-шки с этого можно зарабатывать, поставки DeFi ликвидности.  И нам нужна такая морда на сайте, чисто фронтовская, где можно что-то похардкодить, чтобы кораблики плавали из одной точки в другую.  допустим, из Бразилии в Россию.  Это для чего? Для клиентов или для себя?  Да, да, для клиентов.  Ну, для себя нет никакого смысла, а для клиентов, чтобы просто им весело было.  Потому что поставка едет там, типа, днепро.  не 30 и вот хотя бы проверять какие-то статусы видеть прикольно потому что вообще логистика карга работает так что там очень редко статус обновляются и часто но не актуально и все это больно Мы им решили такую фигнюху сделать, и я это сделал, угадай, за сколько?  Ну, не за 20 дней, с корабликами часа 2, наверное, надо было потратить.  За 15 минут.  Я просто это описал в Bolt New, называется фигня, вот знаешь я, да, все это описал, он мне с первого же раза выдал неплохую фигню.  Я немножко там подкрутил где-то хедер, где-то что-то, и все, и оно работает.  То есть я там буквально… И задеплоил?  Да, и задеплоил.  Там же задеплоил?  Да, да, да. Там же задеплоил. Это просто вообще… вообще снесло мне башку, потому что у меня есть такой бэклог идей, типа, что можно поделать, и года четыре назад мы с Друганом Мы им придумали штуку для благотворительности.  Типа, знаешь, на сайте покупаешь пиксели.  Вот был такой сайт, миллион...  One million dollar site, что-то такое, пиксель.  И чувак продавал миллион пикселей, по одному доллару какая-то такая идея и продал потому он сказал что мне нужны бабки на этот на обучение и продал все пиксели но правда их все вы купили гамблеры всякие быстро стал популярным а это Это бесплатный трафик фактически.  Они выкупили все эти пиксели и там всякие казинохи.  Это крипто какие-то пиксели?  Нет, это сайт из 2005 года.  там вообще, он до сих пор существует, страшно страшный, но задачу выполнил, он собрал миллион долларов, блин.  Первый, по  Прото-крипто-энертиза.  Типа того, да.  Прото-энертиза.  Прото-энертиза.  И мы придумали нечто подобное для...  благотворительности, да, чтобы эти бабки, точно такой же сайтик, покупаешь, у тебя трафик, рекламки, все дела, плюс мы про тебя там рассказываем в социалках, ну, ты платишь, собственно, за Собственно, за пикселя, да, за размещение на этом сайте.  Вот. И мы сами пытались на JS это что-то собрать с моим другом еще вот, типа, пять лет назад, и это было больно, а сейчас я это описываю.  Я это опять же сделал за 15 минут.  Все супер плавно работает, зона выбирается, кошелек, метамаск прикручивается, и ты можешь что-то купить.  Метамаск. Ну, типа, то есть ты натурально можешь привязать к этому сайту метамаск.  Я просто переспрашиваю. Для меня суперактуальный вопрос. Как в моей ситуации принимать платежи?  Потому что у меня банковского счета нет в США, но при этом надо, если я какую-то чистую...  Это инкорпорировано в Европе, да?  Нет, в Деловере.  А, в Делавере.  Меркурий какой-нибудь ты можешь открыть?  Не могу.  Не можешь, да?  Не могу, да.  Я тебе могу сказать.  Могу скинуть. Ты разве не знаешь Данила GoGoWorld или GoGlobalWorld, что-то такое?  Нет, не знаю Данила.  Нет, я тебе сейчас скину. Он прямо целенаправленно этим занимается.  И они открывают счета, и компания, и все такое.  Ну, много кто занимается, но чуваки, которым я заплатил полторы тысячи, компанию открыли, а счет не открыли.  И говорят...  Компанию открыть, конечно,  это за полторы тысячи многовато.  Мне кажется, даже я бы справился дешевле, типа там через какой-нибудь Stripe Atlas за 500 баксов вообще спокойно можно это сделать.  Тогда не заморачивался.  Мне надо было просто, чтобы сделали, а я не отвлекался на всякую ерунду. Ну вот, Данил,  ну, с вероятностью 90% поможет открыть счет. То есть у них прям и сложный кейс, и несложный, но в твоем я не думаю, что что-то сложное есть.  У тебя явно есть ВНЖ какое-то нерусское, да?  Ну, не только российский паспорт.  Я уже на гражданство.  Ну вот, вот.  Да, короче, напиши, что-то должно получиться.  Из того, что новенького, да, на гражданство поднялся.  Поздравляю.  я перед Рождеством пойду узнавать, что по моему гражданству, аргентинскому.  У меня уже год в процессе.  Обычно за это время уже получают.  Вот ровно год стукнет через неделю.  Ну, пока что-то подзамедлили они процессы.  Надеюсь, к какому-нибудь апрелю, что-нибудь такое, маю, на день рождения свое получу.  Будет вообще супер подарочек.  да госсвищнемся на то что я придумал я не трогаю вексу в плане би ту си потому что как бы не чисто все твое, не хочется туда лезть, что-то как-то мешать, думал покрутить ее в B2B, да, вот в этой sales штуке, что мне очень нравится, то есть, типа, real-time помощник, все дела для sales.  Но конкретно этот фондик оказалось, что они инвестируют в B2B, что стало для меня открытием.  Да, да, они инвестируют в B2B, инвестируют только в B2C.  Я, опять же, не сую туда вексу в текущем виде, хотя...  на B2C, да, опять же, потому что, ну, это надо с тобой обсудить в первую очередь, насколько тебе это интересно или неинтересно, потому что там условия достаточно жесткие.  То есть, ну, давай я расскажу.  Спасибо, наверное, да. Спасибо.  Не совсем даже так. У них идея такая, что они берут тебя как сотрудника в этот стартап, который ты придумал.  Это называется вечерная студия?  Да, наверное.  Венчур-билдер, венчур-студио, да.  И поэтому я не думаю, учитывая, что у тебя уже есть какие-то бабки там ангельские, все дела, не думаю, что так надо поступать.  Но расскажу всю суть, чтобы понять, к чему я пришел.  Интересно.  Просто, тем не менее, какие...  Да, про условия.  Короче, да, про условия.  Окей.  Сначала идея там согласовывается, ты ее придумываешь, все эти...  То есть, условно, как PM выступаешь, на первом этапе там что-то запэха в духе 3K за месяц.  Вот. Идею согласовали, какие-то там кастдевы провели, что-то показали.  Мы понимаем, что потенциал у этого есть, и формулируется главная задача.  Главная задача для всех одинаковая – это миллион долларов накопленный в день.  выручки. В идеале примерно за год. То есть за год заработать миллион долларов, тогда считается, что все заебись, и в проект докладывают много денег. Но перед этим, пока оно бежит, нужны же деньги, да? И тебе дают.  Тут получается 3К зарплата плюс 4К на тесты.  Это 7К на первый месяц.  30-45 дней примерно так.  Потом, когда подтверждается...  Гипотеза, что с этим в теории можно работать, тебя берут на ЗП уже постоянно.  То есть вот это первое, это отдельный контракт, все с документами, со всем.  тебя берут на запятом что-то в духе тысяч пять евро ты бежишь до mvp три месяца дается 40 тысяч евро такая логика дальше Ты начинаешь там показывать выручку, все дела.  ЗП потихоньку повышается, точно не помню там, как по стейджам, но опять же тебе по этапам дают.  и перелистнул у тебя пошли пыль заки тебе дают уже там условно счету в духе 100 200 кей на евро на дальнейшее развитие вот на эти папки по моему ты как раз должен дотащить до миллиона долларов накопленной выручки.  В этот момент тебе дают 5% shares.  Это твоя доля, которая не размывается при входе дальнейших инвесторов.  Плюс у тебя...  есть опцион на 4%.  То есть суммарно ты можешь там 9% компании получить, и после миллиона долларов накопленной выручки у тебя зарплата 12-16 тысяч евро в месяц, и тебе кладут на проект от миллиона до десяти, в зависимости от того, чего надо.  Но ты при этом SEO или какая твоя...  Это хороший вопрос, кстати, у меня завтра созвон, с ними уточню, но ты точно лицо какое-то принимаешь в решении, насколько я понял.  минимум по продукту на то есть минимум на что можно рассчитывать как я понимаю пока но я уточню именно по позиции как она там будет назваться это что-то в духе business owner наверное это можно так назвать вот и чем что что они какая у них корпоративная структура что что за компания  Где инкорпорировано?  Никакой тайны в этом нет. Я, по-моему, даже говорил.  Фонд называется SKL. SKL VC. Можешь вот так погуглить.  Сами они, насколько я помню, в Европе, но твоя компания регистрируется в зависимости от того, где она перформит, да, типа где она...  Не помню это слово.  Короче, где ты работаешь, да, где ты работаешь.  Вот, опять же, с учетом там лучшей налоговой ставки, все вот это, вот это проверяем.  и компания инкорпорируется.  Вот.  Ну, слушай, вообще выглядит, я думаю, для многих европейцев просто вообще как прекрасный.  Мне тоже показалось  Это лютой сказкой. Я поэтому и впилился, я за ними слежу уже больше года, но у меня домены, те, которые мне подходят, то есть я к ним долго стучался с крипто, они в крипто вообще не инвестируют.  вот и у них вот такая вот чисто by тусишь на я тема я вот сейчас это новое направление типа sandbox они туда берут уже классных продуктов то есть меня там про собесили все говорить о да Берем, погнали. Я им говорю, типа, что вот у меня домена, другие говорят, ничего, что-нибудь придумаешь.  Вот, и я придумал.  Интересно, если им дать, извини, я перебил.  Интересно, если им дать вексу на оценку, чего они предложат?  Можно поговорить, спросить.  Но с таким B2B вектором векса они меня отшили.  Вот, сказали, что типа, ну, такого, во-первых, много, я представляю, что, естественно, это идея на поверхности, да, но, опять же, зная, что у нас уже есть, ну, у тебя, в смысле, уже есть MVP, И в теории как-то можно было бы договориться, чтобы я там, ну, скажем, что-то B2B делал, вот, это может работать, вот, но они отшли, я говорю, ну, ладно, хрен бы с ним.  Знаешь,  Состояние рынка я бы охарактеризовал, как ты едешь в автобусе вечером, и в нем сидит три человека, остановится остановка, у нее заходит будет еще один или два вот такое состояние рынка и при этом все говорят как тесно как тесно вот меня тоже это удивляет 60 например сейчас  но пока рыбу типа два года вот  Такими темпами, как минимум, еще года три, наверное, можно.  То есть мой продукт до текущего момента, до сегодняшнего дня, когда опубликовался экстеншн.  Откровенно говоря, до конкурентов не дотягивал по ряду причин. Да, возникли конкурентные преимущества, объективные, случайные.  И на этом я просто вижу, что у меня ретеншн, вот как я запустился, у меня там определенное количество пользователей там.  с десяток пользователей еще вот с самых первых самой первой версии  retention что что за конкурентное преимущество я просто кроме fire Блокс, или как он там называется, и FireFiles, и твоего не пробовал ничего.  Ну вот, со слов пользователей исключительно, здесь всякие мои фантазии отставляем.  То, что говорят пользователи. Это первое. Это то, что штука не присылает участника во встречу. Это первое. То есть людей больше всего бесит...  когда на встрече висит там сидит пять человек и 5 аватаров вот именно вот людей не столько напрягается то что кто-то чем просто наличие аватара просто на экране.  Вот это вот бесит.  Вторая история – это мультиязычность.  А кто делает пять аватаров?  Сори, что перебил.  Вот FireFiles вроде бы делает одного.  одного бота засылает, и потом всем можно разослать сам. Логично, но я  просто со слов говорю.  Я сам таких встреч не наблюдаю.  Обычно я вообще один с вексой, а больше никто ничего не знает.  90 процентов это как опять-таки про пустой автобус потребность есть это подтверждается просто потребительское поведение эта потребность очевидно шире, чем сейчас рынок в принципе только ощутил.  Что очень комфортное состояние рынка.  То есть...  Это совсем не какая-то очевидная идея, которую все, знаешь, сожрали. Но со стороны я понимаю, что это выглядит так.  Да, абсолютно верно.  Второе очевидное конкурентное преимущество – это такая адаптивная мультиязычность, я бы так сказал. То есть ты можешь перейти на другой язык, она точно же перейдет… в другой язык.  Чаще всего люди жалуются, что они просто потеряли встречу из-за того, что надо выбрать язык перед студией.  Вот.  там приходит абракадабра, и с этим сделать ничего невозможно.  Прикольно.  Все остальное это мои фантазии и визионерство, на которое тоже, в принципе, Реакция есть, я говорю о том, что мы собираем знания, которые доступны, как в чате GPT, повертые в собственное знание.  Это прикольно, это что-то такое типа...

## Meeting 12 - November 25, 2024 12:54
Topic: Dragging Mechanism

- [00:33] Dmitriy Grankin:  У тебя всё-таки штука, которая перетаскивает оттуда сюда.
- [00:00] Dmitriy Grankin:  смотрим как будет развиваться Записываем встречу, посмотреть что у нас здесь будет.

## Meeting 11 - December 12, 2024 09:59
Topic: Workload and Deadlines

- [00:07] Dmitry Grankin:  Сейчас у вас, наверное, перед Новым годом там вообще завал, да?  Да, да, да, это точно.  Так, что у нас будет еще?  Женя, Женя подожди.
- [03:02] Dmitry Grankin:  Ну вот, я чуть зашла и думаю, а где моя встреча с Егором?  Нет, по-моему, ее тут нет.  нету и пошла в старый а как же такие там нету человек почему-то не знаю вот у меня была наша покажу А, в понедельник вроде. Вот эта встреча, вот я на ней что-то писала.  9 декабря.
- [01:18] Елизавета Иванова:  И, по-моему, я одна сейчас так активно им пользуюсь, но я продюсерам прям стараюсь тоже это все, конечно, брять, потому что удобно.  Но вот у ребят просто не хватает времени погрузиться в это, у продюсеров именно.
- [15:25] Dmitry Grankin:  Давай посмотрим еще разок на новый интерфейс дэшборда.  Так.  Интересно, как...  Сейчас я три штуки сверху.  сверху нажму поделиться да давай какие-нибудь кэшфлоу типа обсуждали вот я хочу понять как прошла встреча про кэшфлоу про кэшфлоу пример вот сюда я могу написать типа и и ноутбэйс summary  Ah, dá?  Ага, поняла. То есть он сразу реагирует. А можно остановить? Или он уже все?  Ага, все, хорошо.  Делай по прошедшей встрече.  и решениях, которые были приняты, например.  А можно вот здесь как-то ее, ага, вот сюда, типа, эту можно вообще закрыть, да, вот так.  вот так а если я хочу чтобы она типа поменьше как-то вот просто клево удобно  Круто!
- [19:06] Dmitry Grankin:  Название основного митинга.  Это типа просто, да, да, я просто Лизе рассказывала именно про проекты, да, да, да, да, да, да, да, да, какая маржа.  там должна быть но мы но мы а и у меня он вырубился в какой-то момент да и перестал писать Ну давай вот с Викой, например.  А вот что-то какие-то встречи пропускаются, потому что мы с Викой были на встречах точно с 8 октября.  А, может, я просто вексу не врубала?  Но с Викой типа встречи...  А, поняла, поняла.
- [04:20] Елизавета Иванова:  это она и есть по идее не это старое что-то маленькие заказы отказы типа да у нас финансовые трудности типа что-то мы обсуждали а вот на это новой, 9-го, мы говорили про там что-то уже про Новый год, про то, что надо Диму переводить в другой отдел, да, и вот такие всякие штуки.  странная история может быть знаешь что у нас повторяющиеся встреча и он как бы не привязал к 9 декабря эту встречу, потому что она каждую неделю.  И он мог взять первичную дату.  Да нет.  А можешь нажать на транскрипт?  Правильно там, ну, вот там такая кнопочка есть, транскрипта.  Какая?  Вот чуть повыше, вот это, да.
- [17:21] Елизавета Иванова:  А вот здесь теперь распиши задачи на основе принятых.  их решений по каждому типа спицу.  Да, да, слушай, все четко.  Да, да, мы типа по кэшфлоу сейчас собираемся и будем какие-то придумывать новые.  Все, мне нравится.
- [25:44] Dmitry Grankin:  Что я не спросил, да, это какие юзкейсы? Ну, вот я Я понимаю, что это внутренние созвоны, это статусы, созвоны с клиентом.  У вас тоже там есть, да?  Да, но вот внутренний статус, это кто кому чего обещал, это понятно, да?  Да, и там много народа.  Остальные все по компашкам поменьше.  Да, а созвоны с клиентом, какие юзкейсы для версии? Что промотят, что спрашивают?  Я не знаю, что они могут спрашивать, но общая идея то, что они там презентуют, например, проект, предложение, вот, либо согласовывают что-нибудь и, соответственно, можно спрашивать.  спросить там, как отреагировал.  То есть, по-моему, такие кейсы делали.  То есть, по-моему, они так поступали.  Вот. Вопрос, на каком этапе?  Вот с командой потом, опять-таки, может, по проекту.  Но это то же самое, как статус.  Вот, и юзкейс передачи дел, когда ты наговариваешь, и там накапливается.  Там накатывается, вот я сегодня запросила посмотреть, как это выглядит.  Просто интересно, как бот передает.
- [21:35] Dmitry Grankin:  А все видят мои вот эти notes? Хорошо, это важно, да.  Я иногда пишу, типа, а что там про Вику?  О, а как это делать?  А, то есть мне транскрипцию не нужна?  Ой, сейчас там будет что-то. Слава Сыркина наед. Простите.  Да, обсуждение, моржа для Дианы. Да-да-да, много чего.  Ну, прикольно.
- [14:15] Dmitry Grankin:  путем митинг-бота, который приходит на встречу в качестве дополнительного чувака.  То есть здесь вот есть трейд-офф.  То есть такой функционал можно допилить.  ...допилить, тогда оно будет надежное и можно даже с телефона заходить на встречу.  Вообще на нее не заходить, вот придет и сам запишет.  происходило.  Это моя мечта.  Вот сейчас вырубился, вот прямо сейчас я сделала действие, и он вырубился. Я зашла обратно с текущей вкладкой, перешла на вкладку Сейчас я вернула и добавилась сверху третья строка.  Вот, я сейчас перешлю давай в наш чатик, как у меня это выглядит.  Вот сверху, видишь?  Я сейчас только мышь, я вижу ролик с мышей.  А я отправила в чат.  А, окей, окей, ладно.
- [00:49] Елизавета Иванова:  Не, делаю, делаю. У меня сейчас недавно был прикольный созвон, на котором прям мне понравился по кнопке, которая там была доступна, по-моему, типа...  Это Your Nodes-Based Summary.  Она очень-очень клево делает все репорты.  И так просто потом допиливаю чуть-чуть, отправляю, это все в задачки превращается, какие-то вещи вообще генерировала даже. Короче, пользуюсь, мне прям очень нравится.
- [00:36] Dmitry Grankin:  придет учета как новый дашборд видела да посмотрела прикольно пока ничего не поняла Ну, ты так записываешь и потом ничего не делаешь или что-то делаешь?
- [06:04] Елизавета Иванова:  вот так а как сделать самаре ну вот живот у тебя вот то что написано да да да да слушай правильно ну прикольно все это прикольно Все, и кого обсуждали.  Все показывают.  Но мне нравится, забавно.  А как это можно теперь с этим что делать?  Ну, вот так же.  тот же самый принцип да как и вот тут да да здесь тоже сам простуд больше всего а можно нажать на какую-то гипер кнопки Я сделаю.  Я сделаю.  Все хотят гиперкнопки и будут гиперкнопки, очевидно.
- [20:23] Dmitry Grankin:  а это наши общие созвоны все не хватает вот этого действительно название что-то бластер на нем были вот люди общий статус каждую неделю Да, да, да, так будет точно удобно.  Ну и, наверное, вот такую вот краткую как бы тему.  Ее можно давать вот тут, вот в пояснялке.  Типа, что было, обсуждали финансовый вопрос, сроки проекта, дальше обсуждение акта и срока работы, бла-бла-бла.  Туда это переносить нужно.  Ну и все, да, супер.  Слушай, Дим, у меня прям 3 минуты, наверное, осталось.  Я потом вернусь, да, как попользуюсь еще.  О, кайф. То есть он к себе получит это в интерфейс тоже. Только встречу.  А, понятно.
- [27:01] Евгения Садовская:  То есть я так поняла, что они просто созванивались, она показывала там условия.  демонстрацию крана, и потом попросила, ей написали регламент.  Мне пока не прислали.  Ну вот, можешь попросить, тебе саму встречу передадут.  А у тебя как не юзкейсы?  У меня в основном внутренние встречи.  Экшн-поинты.  Да.  Только мы все обещаем.  Да.  Ну окей, надо будет еще, если в какой-то момент еще созвониться, очень полезно, очень, вот.  если найдете еще там полчасика вот по поводу юзкейсов поговорить с  самым интересным вот понятно что кнопочки нужно на кнопочки будем делать  Окей, хорошо.

## Meeting 10 - November 26, 2024 16:29
Topic: Transcript Usage

- [06:00] Olga Nemirovskaya:  То есть она очень маленькая по сравнению вот с этим.  А вот этот весь список, он мне вообще не нужен.  Ну, то есть зачем мне вот это вот огромное?  менюшка с тем, чтобы смотреть.  Ну, то есть я не очень понимаю, зачем, например, мне развернуть все встречи.  Вот для чего мне это может понадобиться?  Я вообще не представляю, честно говоря.  А ты пробовала смотреть детали?  Детали – это что такое?  Ну, ты открывала вообще, нет? Вот эти.  штучки ну эти не работают кнопки другие вот эти а да я открывала я кстати по  моему здесь их открывала какой-то момент они У меня работали, и у меня открывались они все.  Я на это посмотрела и очень быстро коллапснула обратно.  А каждую по отдельности...  а вот здесь можно выйти типа транскрипт ну не знаю я ну и и как бы для меня Для меня вот это бессмысленно, потому что это как бы слишком коротко, не то, что надо.  То есть мне все равно нужно больше информации.  А то, что там было, ну как бы эти ключевики, ну окей, я это и так знаю.  а вот какой здесь юзерки юзкейс не понятно то есть я если пойду во встречу я пойду ее искать как бы мне там нужны какие-то вещи которые я либо не помню не нужно либо summary либо я буду фала писать либо мне надо там задачек понаделать либо если там что-то более серьезное то ну соответственно я с этим буду как-то еще ...как-то еще работать, но вот это вот как будто...  Ну, то есть я не знаю, для чего мне это, может быть, надо.  Для фоллоуапа это слишком коротко и как бы куца...  Ну, то есть, короче, не туда, ну, не знаю я, как это использовать, непонятно.  Про что встреча была, я и так по названию помню.  По названию и по людям, то есть мне для этого не нужно, наверное, разворачивать ее, но очень редко нужно разворачивать и смотреть, что там было.  вот да я даже эти details даже не нажимала я как я только вот эти нажимала и все Как-то просмотрела это, не увидела.  Так, что-то у тебя, интересно, не открывается там.  А другую какую-нибудь можешь открыть?  Кого?  Транскрипт?  Детейлс какой-нибудь другой.  Ну вот.  Все почему-то не открывается.  А что должно открыться?  Ну там еще должны теги открыться.  А-а-а.  Я не знаю.  Интересно, почему?  Это не ту версию, что я опубликовал.  Странно.  А может быть ссылку я на какой-нибудь не той ссылке зашла?  Нет, нет, у меня тоже самое.  Ну вот.  Так, и, значит, получается, что много места занимает...  Ну, это же фактически менюшка.  Спицы занимают много места, горизонтально много места.  Ну да.  Окей, то есть...  Ну, то есть, для меня вот это...  Попробуй его, попробуй его.  Смотри, попробуй его перед вот этот, между ними стенка, попробуй ее передвинуть туда левее.  Она мне только в обратную сторону, еще шире делается.  Так, подожди.  Ну, это не значит, что...  Что я тебе предлагаю этим заниматься все время?  Это значит, чтобы понять, какой для тебя будет комфортный размер.  Она у меня не двигается в ту сторону.  Она у меня только шире делается.  Ну, у меня сейчас...  У меня сейчас получилось.  Ну, вообще, да, получается, что ее можно прям сильно-сильно уменьшить.  Не знаю, почему она у меня не уменьшается.  Она у меня, видишь, я ее две...  Бегаю в эту сторону, а она-то двигается в другую.  В чем логика?  Не двигается.  Короче, я не знаю, как это сделать.  Ну окей, понятно.  Ее надо сделать уже раза в два.  Ну, мне кажется, вот какая-то должна быть, просто вместо нее, наверное.  Или, может быть, одну под другой, типа у тебя вначале по спикерам, отдельно по датам, например.  Не знаю, как это.  Или раскрывающиеся, типа хочешь по датам, хочешь по спикерам сортировку у тебя есть.  Ну, я уже говорила, да, что мне не по спикерам надо, мне бы как-то вот по типам встреч было бы намного более...  полезно если было бы разбивка не по спикер ну по типам понятно это я  Это ты...  Я смогу предложить там самой добавлять эти типы. Ну, да, теги.  Вот это было бы даже классно.  Теги добавлять.  Вот. Сам-то я вряд ли эти Теги смогу размечать. Ну, смогу, но потом.  Если какой-то набор тегов будет уже сформирован, то я смогу автоматически предлагать теги.  теги. Да. Ну, а потом на встрече это совсем несложно будет сделать, выбрать тег, например, прям в их старшине.  вот есть здесь идея в чем этого театра как ее донести это по всем встречам и не только по всем встречам а по тем которые Которые выделены, а что значит выделены? Выделены это значит, что если мы взяли какого-то спикера или какую-то группу спикеров, здесь еще можно группу спикеров на сайте делать.  а как это сделать я же здесь вот это что то есть типа я вот так могу выделять их да и они у меня кажется вот и тогда ты сможешь чараться этой  по встречам с этими спикерами.  Я поняла.  Да, короче, вот эта штучка совсем непонятно, что это такое.  Я даже ее не нажимала.  Короче, что это?  Может быть, по умолчанию ставить наоборот вот этот?  формат а сингл как опцию ставить то есть тогда тогда мне кажется будет понятнее Что ты можешь выбрать одного, а можешь выбрать много.  И как бы вот выбираешь, он у тебя выскакивает.  Вот даже так визуально более понятно, чем...  Вот в этом формате, что тебе надо здесь что-то выбирать.  Так у тебя сразу понятно, что надо отжать какое-то количество с этими галочками.  Я бы, короче, по умолчанию вот этот ставила.  А зачем он вообще нужен, вот второй?  Типа вот этот зачем нужен?  Можно же в том...  Всегда существовать? Нет?  Ну, а как ты будешь здесь существовать, да, посмотрим?  Ну, если тебе надо выбрать одного, ты выберешь просто одного.  Если надо выбрать два, ты выберешь одного.  что ты всегда можешь отжать обратно и тогда ну то есть короче но мне прям реально вот так намного более понятно что тебе надо выбирать потому что вот так непонятно что надо выбирать У тебя просто какой-то большой список.  И что ты ещё и можешь много выбрать.  Ну да.  я вот так прям оставил а вот это что за такая большая кнопка  а это старый вид  А зачем она нам здесь нужна?  Ну, мало ли, может, тебе надо.  Пока что.  Старый дашборд.  Ну, ладно.  Пока лучше оставить.  Короче, я бы здесь...  Ну, ты тогда напиши, что это...  Типа просто Old View или что-нибудь такое.  Как это называется там?  Типа старая версия дашборда.  Что такое Legacy Dashboard?  Legacy – не очень понятное слово, наверное.  Не, непонятно.  Вообще, даже немножко страшно, что ты там, куда тебя может унести.  Что это такое?  Legacy – это нато устойчивое.
- [55:00] :  и в этом ворк обязательно должны быть многие штуки.  Запись встреч, потому что это реально...  Мне пришлось искать.  Я перетыкала кучу всего.  Как бы есть проблемы.  Проблема вот эта с тем, что у тебя...  Я поняла, что участник, что мне приходит еще один учитель, не подходит.  Потом следующий стал вопрос про то,  что ты можешь экспортировать транскрипт или нет.  Где-то ты не можешь, потому что где экспортировать транскрипт?  Ты не можешь его забрать.  Смысл мне это дать?  Ну, и тогда какой не нужно тоже?  Ну, и вот как бы третье, это чтобы у тебя это было бесплатно.  То есть я взяла тактику, вот этот я и Джейми,  и в сумме там сколько у них?  25 встреч надо, получается, в месяц.  Зум у меня этим закрывается.  Как? Воих?  Но если бы не было...  Вот так.  Вексы, наверное, мне бы пришлось какой-то из них выбрать,  типа 2 поплатить за это.  Ну, я бы, наверное, выбрала тактику,  потому что у них остается возможность хотя бы во время встречи  потому что я же этим пользуюсь тоже.  Как бы в моменте видеть еще транскрипт.  И это оказывается очень полезно.  Окей, то есть польза существует,  она находится в том, чтобы записывать встречи,  и это уже как бы...  Это уже must have, да?  Это уже как бы не обсуждается, по сути дела, да?  Ну, да.  Вот.  И дальше, собственно,  дальше уже начинается вопрос.  Первый уровень этот ты записала,  взяла транскрипт,  пошла с ним что-то делать, да?  Вот.  И тебе, в общем-то, не важно, то есть ты забираешь транскрипт сразу.  Ты пришла, закрыла, тебе там...  То, что у тебя 100 встреч записалось, и ты не можешь по ним ничего найти в старом дэшборде, для тебя это не проблема, потому что сразу забираешь.  Да, для меня это вообще не проблема.  Потому что сразу забрала и что-то дальше с этим делать.  Ну, я забрала, да.  Но дальше все равно как бы смысл всего этого не в том, чтобы нажать кнопку и успокоиться, да?  То есть ты забираешь транскрипт сразу, ты пришла, закрыла, тебе там то, что у тебя 100 встреч записалось, и ты не можешь по ним ничего найти в старом дэшборде, для тебя это не проблема, потому что сразу забираешь.  Да, для меня это вообще не проблема.  Потому что сразу забрала и что-то дальше с этим делать.  Но дальше все равно как бы смысл всего этого не в том, чтобы нажать кнопку и успокоиться, да, как это, fear of missing out, не в этом, да, а все-таки в том, чтобы обработать информацию, ты ее сразу обрабатываешь.  Да, конечно.  А что ты получаешь на выходе?  Ты делаешь это для того, чтобы проанализировать продукт?  Продумать, получить новые какие-то идеи или для того, чтобы получить какой-то текст, который ты отправишь, например?  У меня и то, и другое я делаю.  У меня как бы я делаю и follow-up, ну вот я сказала, что у меня это либо включается в контекст разговора с клиентом или там с партнером, и я целый чат с ним как бы раскручиваю.  Плюс я, например, делала как? Я выгружала, ну вот, кстати, вот, я выгружала несколько встреч, которые мы проводили, и из них, например, собирала себе транскрипт.  для, не транскрипт, скрипт для новой встречи под конкретного партнера.  То есть я беру информацию о партнере, я беру транскрипт...  нескольких старых встреч похожих и генерирую текст разговора, который мне нужен.  Я забираю себе это там на планирование, на задачки, Анализ какой-то. Что еще я делаю?  Планирование задачки. Анализ, то есть ты чатишься, вытаскиваешь оттуда какие-то...  Экшены, анализируешь, делаешь скрипт звонка, как следующую встречу проводить с другим партнером.  Ну, плюс у меня есть встречи, с которых мне надо сделать какие-то гайды или какие-то доки, тоже вот такое я делала.  Чего еще я делала?  Ну, на самом деле, самая частая история, это вот сейчас в текущем моем флоу, это фоллоуап.  И коммуникация в едином чате, вот это, наверное, самое частое, что у меня есть, и самое удобное.  Так, коммуникация в едином чате.  Смысл в том, чтобы из разных каналов спать информацию в одном месте.  И там...  Ну, это получается, у тебя как бы, у меня нет стандарта, то есть как бы я не, ну, у меня нету типа типовых ответов, да, то есть вот это какой use case, когда?  Ты не можешь подготовить стандартные ответы под клиента?  У меня еще этого нет.  И я не уверена, что это возможно.  Может быть, и возможно.  Вот, но при этом у тебя скорость ответа, она увеличивается, ну, прям в разы за счет того, что у тебя есть единый контекст, и ты как бы в этот контекст постоянно что-то докидываешь.  Скорость ответа. Ответа на что?  Ну, у меня, я провела эту первую встречу с партнером, а дальше он как бы возвращается.  и у него может быть там сто пятьсот вопросов и на эти сто пятьсот вопросов как бы чтобы мне было максимально быстро отвечать вот я делаю это Это чат, где как бы там все единая информация, куда я могу быстро голосом наговорить ответ, и он как бы мне нормально все сформирует.  хорошее сообщение, которое я тут же быстренько закину в чат, обратно.  То есть у тебя какой-то есть там проекция, Когда пришел новая коммуникация с контрагентом, с партнером, ты это кидаешь туда, наговариваешь какие-то дополнительные мысли, И даешь задание сгенерировать ответ.  Да.  Либо я пойду спрошу там какие-то, уточню моменты, мне ответят, я тоже туда же закидываю.  Это что будет? Сообщения в WhatsApp, Telegram, email?  Да, чаще всего это сообщения.  Ну, на самом деле сейчас, потому что поскольку у меня много...  и нерусскоговорящих, и они в основном на почте.  Много на почте, очень много на почте.  Типа 50 на 50 процентов в чатах и на почте.  почте окей а на командных встречах как-то используется ну конечно но вот как бы  Будто бы на командах точно используется, я забираю, потому что у меня там много деталей, которые в моменте ты можешь упустить, и они тебе потом...  Нужны будут, вот я как бы возвращаюсь к каким-то встречам, я могу возвращаться по нескольку раз, потому что мне нужна какая-то конкретная деталь из разговора, и мне приходится...  Идти прямо в транскрипт, потому что он в Summer может не вытащить.  Ну, хотя Клод, в принципе, нормально практически всегда вытаскивает.  Ну, у меня командных встреч сейчас на самом деле немного, то есть у меня прям в разы больше коммуникации с партнерами, а командных встреч, типа, 1 2 в неделю но они такие супер насыщенные а твоему шефу было бы интересно видеть список твоих встреч  с названиями, с самыми вот этими, такими коротенькими, с самыми.  Ну, отчего бы он там понял?  Ну, в смысле, как бы...  Ну, окей, ты можешь зайти посмотреть, что там я с кем-то встречаюсь, да, но у нас CRM-ка стоит, я поставила hub-спот, и все...  Там стараюсь вести в этих хаб-спот сделках, и то, мне кажется, он туда особо не заходит, понимаешь, хотя там даже удобно смотреть.  А уж на встречах, ну, что ты там будешь смотреть?  То есть, типа, самари по встречам?  Не знаю, мне кажется, это...  Мне кажется, это вообще...  Плохо было бы так одним взглядом окинуть хотя бы просто количество встреч, с кем встречается, посмотреть, о чем...  о чем разговор и пойти и дальше пресерчить что-то интересное это вот как раз относить как ты говоришь новое это как раз А если у тебя таких трое, четверо, пятеро человек находится в подчинении?  Все-таки все рамки надо заполнять, и кто их как заполняет, и дело в том, что даже если ты их добросовестно заполняешь, у тебя все равно там не будет этого всего.  Если только это полностью две интегрированные системы, тогда будет. Но здесь как раз речь идет о том, чтобы элемент.  МТСРМ прямо непосредственно реализовать.  Ну, тогда тут надо, наверное, смотреть, как ты можешь это посмотреть.  Ну, вот, может быть.  Список прямо ось такий йде.  Ось, не сортувальний.  Ну, так, з тегами, хто на зустрічі був із співробітниками.  Ну, да, да, наверное.  Думаю уже на тем, как именно для компаний.  доставлять. Ценность, мне кажется, она тут как раз прямо основная ценность где-то здесь находится.  Ну, может быть, да.  Самое интересное на командном уровне.  Ну, мне кажется, что это как бы у тебя все равно, у тебя тогда первое, Что тебе нужно сделать? Тебе все равно нужна интеграция с CRM, потому что тебе как бы, чтобы вот это было не бесполезное лазание по этим встречам, тебе надо четко понимать, что у тебя там, ну, условно, вот в моей ситуации, что вот этот партнер тебе условно...  Приводит плюс 20 клиентов в месяц, а этот пока ничего не генерирует.  Ну, то есть, как бы связка дальше с продажами, потому что без этого...  Ну, а что смотреть на эти встречи, мне непонятно.  Ну что?  Да, побежим.  А то мне за сыном надо мчать.  Давай, давай, спасибо.  Давай, пока, спасибо.

## Meeting 9 - December 03, 2024 11:05
Topic: User Experience with AI Agents

- [06:28] This chunk is part of a conversation between Daniel and Dmitriy Grankin discussing the use of AI agents in online meetings, specifically focusing on the differences in their functionalities, privacy concerns, and user experiences with various transcription tools. Daniel expresses his dissatisfaction with existing agents and shares specific issues he has encountered, while also showing interest in testing Dmitriy's solution.
Daniel:  participating in talks in meetings online meetings and i would say that almost everyone is um is like using any kind of agent uh one thing that i see that is different on your side is that when one thing that bothers me is that when you have like a meeting with five people and everyone has its own agent the agents joins the meeting and there are like five people and another five engines like in the meeting which is creepy to see like all these agents in some way it's kind of transparent because you see that there's there's an ai transcribing the thing in your case like nobody knows that you are transcribing having the thing and like using AI because you it's it's a plug-in on your own machine right so I can see that you are using it which i don't think it's if it is a good thing or a bad thing but just it's it's different it's a different approach so i assume that you are not using the you are using the real time audio on the machine of the user and not the uploaded audio that the meeting has probably this is how other agents do they they can do work afterwards uh while your agent needs to be like active during the meeting uh like the person needs to have it actually also for the privacy  concern we actually do send a message to the meeting chat informing the participants but it didn't work this time for some reason I need to check why is that but yeah we are trying To comply with regulations we need to do it and we have another one single competitor doing the same thing and they also doing it similar way they they send a message to the to the meeting chat and actually it's a competitive advantage over those who are sending bots because because people tend to hate them.  Yes, yes.  Yeah, I would love to use it for a while, to test it, to know, to like feel, how it goes because i would say that i am not that uh happy with it with the agents that i used so far uh one one agent that i was Usar o TLDV tem um problema que não identifica a língua, então eu preciso dizer manualmente que isso é um encontro português, isso é um encontro inglês.  And this is so bad because it's after the meeting, if it's not in English, it sends an email with a completely crazy, uh, uh, yes.  abstract of what happened and it's so weird yes having and i can't change the language after the meeting happened so it's it's pretty bad so it's not working for me and There is the other Fireflies, I think it's Firefly the name.  Yes.  Yes.  Something like, which is a little bit better dealing with languages, but has some other issues that I didn't like it as well.  Which ones are those?  Yeah, the Firefly, Firefly flies door AI.  Yeah, the fireflies.ai.  Yes.  Let me go into my...

## Meeting 8 - November 26, 2024 15:01
Topic: Knowledge Sharing

- [05:48] Dmitry Grankin:  Мы строим, мы собираем знания напрямую из звонков в Google Meet и даем к ним доступ во время звонка и...  после звонка. И объединяем команды, объединяем эти знания, чтобы они были доступны участников команды вот сейчас существует несколько месяцев с летом вот есть некоторое количество индивидуальных пользователей я хочу привлечь некоторое количество Команд, каких-то бизнес-единиц, которые могут быть в рамках какой-то крупной компании или самостоятельной бизнес-единицы.  единиц вот для того чтобы до для того чтобы выходить на битуме Вот сейчас пока никакой у меня монетизации нет, потому как там есть вопросы с открытием счета, да и пока просто в принципе разработка.  Работка не дошла до того, чтобы прикрутить оплату, просто-напросто, вот, вот, но база пользователей какая-то есть, мы хотим ее расширять.
- [04:17] Dmitry Grankin:  тогда чуть ближе к моему проекту смотри что я делаю я сейчас В очередной раз запускаем продукт, который представляет собой...  Ассистент для звонков Google Meet в крайнем времени делает транскрипцию, подчеркивает разные...  ключевые слова здесь можно его дальше что-то спрашивать и там после этого есть дэшборд который я сейчас как раз раз публикую вот то есть после этих всех встреч мы попадаем в такую систему, где у нас все встречи есть с фильтрацией, попользоваться по спикерам. И можно посмотреть посмотреть значит разную информацию в виде Значит, summary, вот, и почататься со звонком или прям со всеми звонками, или с какой-то группой звонков почататься.  выбрать какого-то пользователя или группу пользователей, почататься, что там у нас с ними было, о чем договаривались и так далее.  Вот. То есть...
- [00:00] :  Да, нормально.  Купер. Тогда рада знакомству. Да, если удобно, тогда можем на «ты», да?  Да, можно.  Купер.  Спасибо, что согласился на встречу.  Всегда приятно, когда люди идут на контакт, независимо от того, вообще как это в дальнейшем сложится.  Вот всякие бывают сотрудничества, партнерства, Поэтому никогда не бывает ненужных связей, как по мне.  Тогда предлагаю построить диалог в таком формате, что я расскажу о себе.  Успеем чуть больше о релевантном опыте относительно твоего проекта.  Ты также расскажешь про свой проект.  И уже, я думаю, мы пройдемся по формату такого вопроса.  Давай.  Окей.

## Meeting 7 - November 25, 2024 21:24
Topic: Планирование действий

- [02:52] :  что мы будем делать с этим  Отход до 254 встречного сапога.  нельзя  занимаюсь своими делами
- [02:09] Dmitriy Grankin:  Ничего, она закрылась-то. Для активных встреч мне никак не обрабатывать. В чем смысл активных встреч?  Что-то в том...

## Meeting 6 - December 03, 2024 14:31
Topic: Психотерапия и онлайн-консультации

- [02:46] Aleksandr Lavrinenko:  поймешь почему я тоже обратил на это внимание но мы не может пока другим путем решили сдвинуть так пока что мы с тобой встречались разреза проекта болтай он все еще работает но там есть сложности с продажами болты я этот к пайла для продукт менеджеров мы сейчас ищем market fit Очень сильно. Пока у меня остались ресурсы. Я там с еще одним нашим знакомым решил протестить гипотезу. У него какая есть? У него есть очень интересная аудитория – психотерапевты.  и у психотерапевтов есть потребность в том чтобы важные вещи по клиента сохранять запоминаете уметь ним возвращаться чтобы трекать прогресс у них там есть прям четко и при При этом у них есть особенность аудитории, что это приватно, то есть это не должно быть никак визуально заметно для кого-то.  А у них не очень высокое компьютерное грамотность у большинства.  и для них они там используют инструменты с которыми начали словно начал брать делать заметки google docs ты будешь продолжать делать google docs до остатка всех своих сессий такое мышление и но при этом они поскольку у них Проблема – это актуально, они хотят попробовать все решить, и тут мы вылезаем, и такие вот у нас софт, который позволяет…  Да, извини, уточню, это онлайн-консультация?  Да, это онлайн-консультация.

## Meeting 5 - December 02, 2024 15:30
Topic: Work Progress and Coding

- [00:12] Dmitriy Grankin:  Сейчас вылезаю из этого, из этой норы.  сейчас перед встречей посидел покопался в этом вашем фон или несколько Мог его закончить, наверное, не предполагалось, да?  Что значит не предполагалось?  Что его можно до конца пройти.  А что он не смог закончить?
- [00:00] :  Да, Игорь, привет.  Что, как дела?  Ничего, вот сидел несколько недель, кодил.
- [19:25] Dmitriy Grankin:  У них есть название, у них есть коротенькие и емкие summary, у них есть теги, где...  видишь да да вот где можно кликнуть и на ключевые аспекты он там что-то вот можно по просто спрашивать все что угодно про эту встречу вот или там делать самое можно что например удобно мы что-то про коздел говорили Вот так сделать и что-нибудь про козлев получить.  Вот такая штука.  Вот. И ну есть что очень удобно по спикерам на самом деле все дело выбрать спикеров вот и такая штука у меня То есть, в принципе, Есть ассистент по всему массиву звонков или по селекции из того, что мы здесь выбрали?  Слушай, ну как бы норм.  Ну, что, я не знаю, давай тысячу, я там не знаю, ты, ты, как ты хочешь сейчас от меня, типа, что, видите, я там, не знаю, заплатил, можем заплатить, или там, какую-то обратную.
- [10:11] Igor Lyubimov:  У меня еще клиентские такие звонки, они все, ну, типа, однотипные.  То есть там мы частным там скинем эту ссылку, скинем ту ссылку.  ссылку. Типа вот зарегистрируемся или мы там подумаем, через неделю вернемся.  Вот. И все.  А я на следующий день уже забываю все, что мы там обсуждали.  И вот поэтому я иду в тактик и возвращаюсь, и пытаюсь прочитать.  Транскрипт.  Да, прочитая транскрипт, он не всегда попадает, а еще вот когда язык не переключается.
- [09:10] Igor Lyubimov:  То есть там есть какие-то 10 AI-кредит у тебя.  И ты как бы на один мид применяешь этот AI-кредит, и дальше там можно summary сделать, можно action-пойнты, можно full-up написать, можно, ну, короче, разные вот акценты.  активности можно сделать с текстом, который он обгенерил и это все за один кредит как бы будет. Соответственно 10 таких методов. Вот время от времени я делаю Но нет желания туда сильно возвращаться.  Иногда вот как-то захожу, но не попадает.  но я бы хотел я бы хотел наверное какой то вот сейчас до сказал слушать туда  какой-то флоп я бы блин хотел по каждой встрече чтобы мне там да вот какой-то самый Смотри, типа, что я должен сделать, например, какой-то вот флоуап.
- [23:05] Igor Lyubimov:  звонки с 1 делаю но у меня в уходе там customer success менеджер и И вот, то есть он будет на всех звонках, он будет учиться там и продажам, и типа там те договоренности, которые мы сделали на звонке, он будет выполнять, типа выставить счет.  Вот, типа, создать чат, типа, еще что-то, тут договорились, тут не договорились, и, соответственно, ну, мне надо видеть, какие задачи, а ему их выполнять.  то есть в идеале это когда ему просто падают твои звонки в идеале наверно когда он видит задачи  Ну, да, там, не знаю, в Slack можно, например, у нас будет общий с ним Slack, в Slack прислать список, ну, просто список, task.  да и там я могу в комментариях написать типа тут правильно тут неправильно типа вот это возьми не возьми ну а дальше уже неважно там вношу перенести тут как вот эта автоматизация интересует скорее типа просто список экшенов получить сразу после звонка ну и кайф  так у нас там во время плюс минус подходит концу а вот ну давай я понял что ты уже прям сейчас Сейчас будешь пользоваться просто потому, что...  Ну да, да-да-да.  Ну то есть у меня...  Ну-ка, подожди.  Подожди, сейчас я обновлю страницу.  ага это ты сам установил так он пишет вершин 923 и за аудит ну пускай это пока будет так  написано до того момента как это у тебя не обновится не обращаю внимание все равно у тебя вспомнил а так а тут  Так, понял, ну ладно, короче, буду пробовать, да, все

## Meeting 4 - November 25, 2024 19:45
Topic: Execution vs Idea

- [28:31] :  идея а совсем другое дело исполнение  совсем другое дело  Ну че теперь, TX0?  чем те, по которым не было обновлений в течение...  Это все та же встреча.

## Meeting 3 - December 05, 2024 19:00
Topic: Brazilian Permanent Residence Research

- [00:43] Dmitry Grankin:  Actually, I've been now researching on how to obtain Brazilian permanent residence, believe me or not.  Oh, really?  are you planning on going there to um actually uh i like um collecting travel documents like For a Ukrainian-born Russian citizen, it becomes a necessity to collect different documents.  a portuguese child and so okay you technically obtain the brazilian citizenship quickly so it feels like an interesting interesting game
- [00:05] Lara Vargas:  I'm on my iPad, so I just want to check on Lisbon.  Yeah, I am. Are you as well?  Yes, on Belas Clube do Campo, not far from Lisbon.  Oh, okay, nice. It's been very cold, isn't it?  for a brazilian like me where uh what part of brazil are you from i'm from rio so yeah i'm not used to this I think the greys is already a lot for me, I mean.
- [04:54] Dmitry Grankin:  sometimes in the future we'll do that because because because why not actually yeah okay um so we we've done something We've done some significant refurbishment of Yexa recently and Now it's not only the real-time and as a real-time assistant but also it's nice it's a nice dashboard and the idea behind that is that you share a lot of valuable insights during these meetings and people tend to share it with you openly because like people people people say things that they tend not to to write each other right and the idea is to collect this and build knowledge with it so each meeting becomes a unit of knowledge with Wow with actions with with knowledge points, discussion points, people mentioned companies with descriptions and chat like that, which enables you to research specific meetings like it's pretty weird why is that in Russian by the way this because we spoke english i believe  but maybe you i don't know i don't know if you and alia were in the call before Okay, maybe, yeah, we got you.  just a little bit of conversation in russian maybe that was what that's the reason and that the previous conversation like action points from that table so just to record recall what we've been up for the follow-up call content calendar yeah so that's how it works and we also have A global chart, which works like what is VEXA.  Oh, okay.

## Meeting 2 - December 05, 2024 13:59
Topic: User Acquisition Strategies

- [09:07] Sunny Wyrzykowski:  Have you tried to, I know you mentioned that you right now, it's mostly just word of mouth.  Have you tried?  to you know do some outreach or other kind of ways of of getting more clients or have you  just kind of focused on word of mouth for now yeah but we've tried product product hunt we've been semi-active on linkedin basically responding to incoming outreach doing some outbound outreach just just little bit uh we tried um we've dry we've tried influencer uh integration with the with it with micro influencer in Brazil and that was quite successful nice well it looks like we are getting those users as a result of that activity  activity now from brazilians from brazil uh-huh so when you mentioned you've tried you know like linkedin a little bit of linkedin outreach and stuff like that has that worked out pretty well for you or um how has that kind of worked out so far  um you know it's a kind of natural uh natural behavior to to communicate with people that are willing to communicate with you and there's nothing really and no deliberate action taken about that and on those things we tried as a deliberate action didn't work that well maybe because It wasn't done in the right way, like automated outreach that looked pretty silly to be honest with you.  Mm-hmm.
- [07:46] Dmitry Grankin:  thousand paying users each paying 20 bucks monthly uh teams teams i mean like uh the product is uh obviously helpful for individuals but it will deliver more value for collective knowledge access and collection with the share uh uh for features uh for sharing that i've been now testing sharing uh functionality which i also and the link to you after the call, so you could basically see what it looks like.  And if you continue using VEX afterwards, you'll, have this call as a as a unit of that uh knowledge cool and then  Right now, where are you kind of at with I know you mentioned you would like to be, you know, in six to 12 months if you'd like to be at, you know, a few thousand paying users, what are you at right now? And have you encountered any any blockers?  keeping you from reaching that call the blockers now is that I don't have bank account and I don't have paint integration just because So that's all completely free for now and now I have something about 300 users which just a little but with a good retention with some of them 500 recorded calls 200 recorded calls So that's like months of daily use.  Sweet.
- [24:26] This chunk occurs towards the end of a conversation between Dmitry Grankin and representatives from Ace Makers, where Dmitry discusses the advantages of his product, Vexa, in comparison to another product, Fathom. He highlights Vexa's unique features, such as its silent operation during calls and multilingual capabilities, while also emphasizing its ability to build collective knowledge from meetings.
Dmitry Grankin:  Okay, so you are absolutely happy about this product nothing really bothers you okay cool that's one of a few products with which i i hear i hear that okay great i i encourage you to to try out vexa because um this what are the competitive advantage on which people are telling me not what I am telling people, but my users are telling, is that it stays silent on the call, doesn't participate.  As a meeting participant, there's some...  noise people i don't know why actually but as a matter of fact and as i mentioned earlier it's multilingual and it's seamlessly um switching from one language to another and as what I'm telling the users you know they are telling me is that we are building knowledge with that so we can research not just a specific meeting after a call, but we can actually ask general questions about who are our competitors, what users say about us.  what we've been doing the last two weeks and yeah creating reports that are derived from the full body of uh collective knowledge how how does it uh how does it sounds to you

## Meeting 1 - December 03, 2024 13:01
Topic: Follow-up and Communication

- [13:35] Anna Pavlova (Lupenko):  Тогда напишите мне, пожалуйста, свой Телеграм, я напишу, скину туда информацию по встрече, вот, и на почту тоже кину приглашение на мероприятие, вот.  Вот, а там же кину потом в телеграм-контакт Александра, чтобы вы друг друга нашли.  Я, может быть, тоже буду, постараюсь быть.  Ну, ладно.  Ну, класс, рад был познакомиться. Давайте так.  Да, взаимно. Да, спасибо. До скорой встречи. До свидания.  Куда кинуть, куда кинуть? Я, может быть, прямо здесь сюда кину, чтобы?  да можно или в линкедин где мы общались целом тоже можно хорошо такой  такое количество сообщений после веб-саммита  неразобранных, что я подозреваю,  что у вас еще хуже.  Это точно. Но  Но приходится отвечать в рамках своих обязанностей.  Да, потому что я после веб-сайта ушел кодить, и там вообще прям совсем...  прям совсем ну ладно а я тогда сейчас кинул телеграм-канал договорились я сразу отвечу с
- [02:37] Dmitry Grankin:  делает до коротенькая делает так там то что нужно вытаскивать самостоятельно вытаскиваете уже в чатике вот собственно еще способы пользовательскую базу наращивать кто у вас есть команде кто нам нужен в команде сейчас нету ищите партнера Нет, нет, никого не ищу. Я соло-фаундер, привлек инвестиции, потратил большую часть. Пользовательская база.  база растет сама собой потихонечку вот конечно надо чтобы она росла быстрее вот но на самом деле продукт объектива конкурентно способен есть конкуренты вот которые хорошо себя чувствуют чувствуют. Они чуть-чуть раньше начали. Вот. И есть хороший ретершн. И много людей там, не так много, Несколько десятков людей, которые несколько месяцев пользуются Павлом Томовым.  И так несколько месяцев.
- [01:01] Dmitry Grankin:  в процессе нужно поресерчить то о чем мы говорим после звонка это выглядит такой у нас У нас открывается дэшборд, которым есть звонки...  ну вот это наш звонок разные предыдущие звонки и самаре здесь можно почататься со всеми звонками  они сразу или с каким-то конкретным здорово после нашего знака скиньте мне пожалуйста результат который получится теперь Интересно посмотреть, может быть кому-то посоветую. В смысле summary, который он делает?


In [64]:
input = count_tokens(input_context)

In [65]:
input* 0.15/1000000

0.00454935

In [66]:
from core import generic_call,system_msg,user_msg


In [67]:
system_prompt = """
you are helpful assistant that is answering user requests by interpreting  while interpreting the context
You have to rely on the context to answer the user request

User is Dmitry Grankin, Address his as YOU and distinguish them in in the provided context in in the response.

Alsay supply reference to specific meeting with [#meeting_id]
"""




In [68]:

r = await generic_call([system_msg(system_prompt),
                        user_msg(f'context: {input_context}'),
                        user_msg(f'request: {q}')])
r



To collect the tasks and action points that you promised to do and those that others promised to you, let's summarize the relevant points from the meetings:

### Tasks and Action Points Promised by You:
1. **Meeting 26**: You mentioned wanting to set user preferences for summaries and actions in the product.
2. **Meeting 24**: You discussed the need to find value in integrating with calendars and sending follow-ups to users.
3. **Meeting 22**: You indicated that you would share the dashboard and its functionalities with others.
4. **Meeting 21**: You mentioned wanting to provide feedback on the meeting page features.
5. **Meeting 19**: You expressed the need to reach out to people regarding project management.

### Tasks and Action Points Promised by Others:
1. **Meeting 27**: Tatiana Sukhova mentioned she would provide UX improvements and ideas for the project.
2. **Meeting 26**: There was a discussion about user preferences, and others were expected to provide feedback on that.
3. **

"To collect the tasks and action points that you promised to do and those that others promised to you, let's summarize the relevant points from the meetings:\n\n### Tasks and Action Points Promised by You:\n1. **Meeting 26**: You mentioned wanting to set user preferences for summaries and actions in the product.\n2. **Meeting 24**: You discussed the need to find value in integrating with calendars and sending follow-ups to users.\n3. **Meeting 22**: You indicated that you would share the dashboard and its functionalities with others.\n4. **Meeting 21**: You mentioned wanting to provide feedback on the meeting page features.\n5. **Meeting 19**: You expressed the need to reach out to people regarding project management.\n\n### Tasks and Action Points Promised by Others:\n1. **Meeting 27**: Tatiana Sukhova mentioned she would provide UX improvements and ideas for the project.\n2. **Meeting 26**: There was a discussion about user preferences, and others were expected to provide feedback on